In [25]:
import pandas as pd
import numpy as np
from pathlib import Path
import selenium
from datetime import date
import sys
sys.path.append('..')
from src.etl import *
from src.data.update_data import *
from src.features.generate_elo_ratings import *
from src.prep_data_for_final_model_new import *
import optuna
%load_ext autoreload
%autoreload 2
import joblib
from src.models.model_preparation import get_draftking_lines, clean_draftking_lines


from sklearn.multioutput import MultiOutputRegressor
import lightgbm as lgb



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
Path.home().joinpath('NBA_Model_v1')

WindowsPath('C:/Users/Jordan Nishimura/NBA_Model_v1')

1) Update Data
2) Preprocess and reload into SQL DB
3) Pull Updated Preprocessed Data from SQL DB
4) Train Model on fully updated data
5) Pull Days Matchups
6) Predict on Days Matchups

## Update Data

In [3]:
db_path = Path.home() / 'NBA_Model_v1' / 'data' / 'nba.db'
season = 2022
# update_all_data(db_path=db_path, season=season)



## Preprocess and Reload into SQL DB

In [73]:
%run ..\\src\\etl.py

Loading raw team boxscore data from sql database...
Loading betting data from sql database...
Cleaning Data...
Merging Boxscore and Betting Data...
Aggregating over last 5, 10, and 20 game windows
adding rest days
creating matchups between Home and Away team aggregated stats
Resorting by date
dropping nulls
loading table back into sql db as team_stats_ewa_matchup


In [4]:
def season_to_string(x):
    return str(x) + '-' + str(x+1)[-2:]

def get_training_data_all(target, con):   

    df = pd.read_sql('SELECT * FROM team_stats_ewa_matchup', con=con)
    df = df.drop(columns=['index'])

    df = df.sort_values('GAME_DATE')

    df = df.dropna()

    columns_to_drop = ['SEASON', 'HOME_TEAM_ABBREVIATION', 'GAME_DATE', 'GAME_ID', 'MATCHUP',
                        'HOME_HOME_GAME', 'HOME_TEAM_SCORE', 'HOME_ML', 'HOME_SPREAD',
                        'HOME_ATS_DIFF', 'HOME_TEAM_COVERED', 'HOME_POINT_DIFF',
                        'HOME_WL', 'AWAY_ML', 'AWAY_TEAM_SCORE',
                        'HOME_PTS_L5', 'HOME_PTS_L10', 'HOME_PTS_L20',
                        'HOME_PLUS_MINUS_L5', 'HOME_PLUS_MINUS_L10', 'HOME_PLUS_MINUS_L20',
                        'HOME_NET_RATING_L5', 'HOME_NET_RATING_L10', 'HOME_NET_RATING_L20',
                        'HOME_POSS_L5', 'HOME_POSS_L10', 'HOME_POSS_L20',
                        'HOME_PTS_opp_L5', 'HOME_PTS_opp_L10', 'HOME_PTS_opp_L20',
                        'HOME_PLUS_MINUS_opp_L5', 'HOME_PLUS_MINUS_opp_L10', 'HOME_PLUS_MINUS_opp_L20',
                        'HOME_NET_RATING_opp_L5', 'HOME_NET_RATING_opp_L10', 'HOME_NET_RATING_opp_L20',
                        'HOME_POSS_opp_L5', 'HOME_POSS_opp_L10', 'HOME_POSS_opp_L20',
                        'HOME_REB_L5', 'HOME_REB_L10', 'HOME_REB_L20',  
                        'HOME_REB_opp_L5', 'HOME_REB_opp_L10', 'HOME_REB_opp_L20',       
                        'AWAY_PTS_L5', 'AWAY_PTS_L10', 'AWAY_PTS_L20',
                        'AWAY_PLUS_MINUS_L5', 'AWAY_PLUS_MINUS_L10', 'AWAY_PLUS_MINUS_L20',
                        'AWAY_NET_RATING_L5', 'AWAY_NET_RATING_L10', 'AWAY_NET_RATING_L20',
                        'AWAY_POSS_L5', 'AWAY_POSS_L10', 'AWAY_POSS_L20',
                        'AWAY_PTS_opp_L5', 'AWAY_PTS_opp_L10', 'AWAY_PTS_opp_L20',
                        'AWAY_PLUS_MINUS_opp_L5', 'AWAY_PLUS_MINUS_opp_L10', 'AWAY_PLUS_MINUS_opp_L20',
                        'AWAY_NET_RATING_opp_L5', 'AWAY_NET_RATING_opp_L10', 'AWAY_NET_RATING_opp_L20',
                        'AWAY_POSS_opp_L5', 'AWAY_POSS_opp_L10', 'AWAY_POSS_opp_L20',
                        'AWAY_REB_L5', 'AWAY_REB_L10', 'AWAY_REB_L20',
                        'AWAY_REB_opp_L5', 'AWAY_REB_opp_L10', 'AWAY_REB_opp_L20']

    X_train = df.drop(columns=columns_to_drop)
    y_train = df[target]

    return X_train, y_train


In [6]:
target = ['HOME_TEAM_SCORE', 'AWAY_TEAM_SCORE']
db_filepath = Path.home().joinpath('NBA_model_v1', 'data', 'nba.db')
connection = sqlite3.connect(db_filepath)

X_train, y_train = get_training_data_all(target = target, con=connection)

In [129]:
X_train.shape

(10955, 566)

In [7]:
X_train

,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_AST_L5,HOME_STL_L5,...,AWAY_TS_PCT_L20,AWAY_TS_PCT_opp_L20,AWAY_EFG_PCT_L20,AWAY_EFG_PCT_opp_L20,AWAY_AST_RATIO_L20,AWAY_AST_RATIO_opp_L20,AWAY_TOV_PCT_L20,AWAY_TOV_PCT_opp_L20,AWAY_PIE_L20,AWAY_REST
0,31.518241,66.650047,11.148672,22.113305,21.954930,25.266237,14.512977,33.514702,23.799377,6.896403,...,0.571432,0.580626,0.514710,0.500641,24.305035,23.174242,12.574422,14.664175,0.503846,3.0
1,29.772751,66.803971,8.735208,19.678742,19.213618,27.270855,15.839389,36.529798,20.015921,9.338738,...,0.622182,0.568204,0.545240,0.483274,24.820241,21.489581,15.404898,17.281397,0.555548,1.0
2,34.273504,69.759524,3.891372,17.806949,20.125979,27.945182,11.645941,38.028180,20.752197,7.152998,...,0.585741,0.554306,0.514862,0.476606,23.658670,20.646725,14.860777,13.379923,0.514737,2.0
3,33.390751,69.422949,6.394982,23.334510,13.025644,19.490081,12.977680,33.674126,21.252429,6.894249,...,0.574779,0.545381,0.498716,0.472069,26.011999,25.555622,12.926552,13.817702,0.530071,2.0
4,34.559555,75.184094,7.061383,17.070850,19.556235,24.876820,13.514408,32.649867,21.339324,8.288540,...,0.594631,0.557912,0.523593,0.489991,24.703460,21.055164,13.115250,12.220492,0.544858,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11130,30.839890,55.931314,13.963104,39.202223,15.084226,20.935176,12.408672,36.818471,28.085088,6.360138,...,0.571530,0.604003,0.519164,0.537806,23.986053,25.305062,12.490602,10.934691,0.462699,2.0
11131,28.319995,53.446985,9.077490,32.248068,19.742139,24.335035,9.580074,29.604762,20.959513,9.964560,...,0.624575,0.571598,0.544119,0.513915,25.897895,23.042860,13.922856,11.631008,0.526891,2.0
11132,26.064455,54.380250,14.284824,36.813042,16.524163,22.847870,12.760495,32.040667,21.118478,6.279447,...,0.599726,0.589269,0.530410,0.527257,24.906112,24.563309,12.030369,11.610109,0.518011,1.0
11133,27.135101,52.847133,13.956899,38.734072,17.435033,24.316039,12.304527,31.447510,23.785239,6.751287,...,0.631192,0.606788,0.569470,0.539957,25.767083,25.782486,13.463299,11.827188,0.505131,3.0


In [27]:
# add_elo_ratings(df)
conn = sqlite3.connect(db_path)
df = pd.read_sql('''SELECT * from team_stats_ewa_matchup_prod''', conn)
conn.close()

In [29]:
df.isnull().sum().sum()

0

## Generate Elo Ratings

In [7]:
conn = sqlite3.connect(db_path)
df = load_team_data(conn, start_season=2013, end_season=2022)

In [8]:
pd.read_sql('''SELECT name FROM sqlite_schema WHERE type='table' ORDER BY name''', conn)

,name
0,moneylines
1,player_tracking_boxscores
2,spreads
3,team_advanced_boxscores
4,team_basic_boxscores
5,team_scoring_boxscores
6,team_stats_ewa_matchup
7,team_stats_ewa_matchup_dev
8,team_stats_ewa_matchup_prod
9,team_tracking_boxscores


In [17]:
# add_elo_ratings(df)

df = pd.read_sql('''SELECT * from team_basic_boxscores''', conn)


In [23]:
df['HOME_GAME'] = df['MATCHUP'].str.contains('vs.').astype(int)
df['SCORE_team'] = df['PTS'] 

In [24]:
df.loc[(df['MATCHUP'].str.contains('POR')), ['SEASON', 'GAME_DATE', 'MATCHUP', 'HOME_GAME', 'SCORE_team']]

,SEASON,GAME_DATE,MATCHUP,HOME_GAME,SCORE_team
12,2000-01,2000-10-31,LAL @ POR,0,96
13,2000-01,2000-10-31,POR vs. LAL,1,86
50,2000-01,2000-11-02,PHX vs. POR,1,108
51,2000-01,2000-11-02,POR @ PHX,0,82
80,2000-01,2000-11-04,SEA vs. POR,1,90
...,...,...,...,...,...
56605,2022-23,2022-11-07,POR @ MIA,0,110
56634,2022-23,2022-11-09,POR @ CHA,0,105
56635,2022-23,2022-11-09,CHA vs. POR,1,95
56658,2022-23,2022-11-10,POR @ NOP,0,106


In [10]:
add_elo_ratings(df)

progress:: 0it [00:00, ?it/s]


KeyError: 'SCORE_team'

In [77]:
#load study with best hyperparameters
study_name = str(Path.home().joinpath('NBA_model_v1', 'models', 'hyperparameter_tuning', 'LGBMRegressor'))    
storage_name = "sqlite:///{}.db".format(study_name)

study = optuna.load_study(study_name = study_name, storage = storage_name)

params = study.best_params
print(params)

# instantiate model with hyperparameters
lgbr_model = MultiOutputRegressor(lgb.LGBMRegressor(**params))

## train model on full data

lgbr_model.fit(X_train, y_train)




{'boosting_type': 'gbdt', 'colsample_bytree': 0.4960110708575806, 'learning_rate': 0.10511063149508226, 'max_depth': 58, 'min_child_weight': 0.947131341685127, 'num_leaves': 11, 'reg_alpha': 0.36636398433425144, 'reg_lambda': 9.061364427698557, 'subsample': 0.8107222779801828}


MultiOutputRegressor(estimator=LGBMRegressor(colsample_bytree=0.4960110708575806,
                                             learning_rate=0.10511063149508226,
                                             max_depth=58,
                                             min_child_weight=0.947131341685127,
                                             num_leaves=11,
                                             reg_alpha=0.36636398433425144,
                                             reg_lambda=9.061364427698557,
                                             subsample=0.8107222779801828))

In [78]:
#load study with best hyperparameters
from sklearn import pipeline
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler




study_name = str(Path.home().joinpath('NBA_model_v1', 'models', 'hyperparameter_tuning', 'SGDRegressor_ScorePredictor'))    
storage_name = "sqlite:///{}.db".format(study_name)

study = optuna.load_study(study_name = study_name, storage = storage_name)

params = study.best_params
print(params)

# instantiate model with hyperparameters
sgd_model = MultiOutputRegressor(Pipeline([('scaler', StandardScaler()),
                                           ('sgd', SGDRegressor(**params, max_iter=10000))
                                           ]
                                          )
                                 )

## train model on full data

sgd_model.fit(X_train, y_train)



{'alpha': 0.035149206342098345, 'epsilon': 5.05778119284027, 'l1_ratio': 0.949791773845651, 'loss': 'huber'}


NameError: name 'Pipeline' is not defined

In [313]:
HOME_TEAM = 'SAS'
AWAY_TEAM = 'LAL'


In [379]:
import numpy as np
import pandas as pd
import sqlite3
from pathlib import Path
import warnings
from datetime import date

def load_team_data(conn, start_season, end_season):
    """Loads basic, advanced, and scoring boxscores 
    from sqlite db and merges them into one dataframe"""
    
    start_season = season_to_string(start_season)
    end_season = season_to_string(end_season)

    basic = pd.read_sql("SELECT * FROM team_basic_boxscores", conn)
    adv = pd.read_sql("SELECT * FROM team_advanced_boxscores", conn)
    scoring = pd.read_sql("SELECT * FROM team_scoring_boxscores", conn)
    tracking = pd.read_sql("SELECT * FROM team_tracking_boxscores", conn)

    basic = basic.loc[basic['SEASON'].between(start_season, end_season)]
    basic[['GAME_ID', 'TEAM_ID']] = basic[['GAME_ID', 'TEAM_ID']].astype(str)
    adv[['GAME_ID', 'TEAM_ID']] = adv[['GAME_ID', 'TEAM_ID']].astype(str)
    scoring[['GAME_ID', 'TEAM_ID']] = scoring[['GAME_ID', 'TEAM_ID']].astype(str)
    tracking[['GAME_ID', 'TEAM_ID']] = tracking[['GAME_ID', 'TEAM_ID']].astype(str)

    df = pd.merge(basic, adv, how='left', on=[
                    'GAME_ID', 'TEAM_ID'], suffixes=['', '_y'])
    df = pd.merge(df, scoring, how='left', on=[
                  'GAME_ID', 'TEAM_ID'], suffixes=['', '_y'])
    
    df = pd.merge(df, tracking, how='left', on=['GAME_ID', 'TEAM_ID'],
                  suffixes=['', '_y'])
    

    df = df.drop(columns=['TEAM_NAME_y', 'TEAM_CITY',
                          'TEAM_ABBREVIATION_y',
                          'TEAM_CITY_y', 'MIN_y',
                          'FG_PCT_y', 'AST_y'])
    
    return df


def clean_team_data(df):
    """This function cleans the team_data
    1) Changes W/L to 1/0 
    2) Changes franchise abbreviations to their most 
    recent abbreviation for consistency
    3) Converts GAME_DATE to datetime object
    4) Creates a binary column 'HOME_GAME'
    5) Removes 3 games where advanced stats were not collected
    """
    df = df.copy()
    df['WL'] = (df['WL'] == 'W').astype(int)

    abbr_mapping = {'NJN': 'BKN',
                    'CHH': 'CHA',
                    'VAN': 'MEM',
                    'NOH': 'NOP',
                    'NOK': 'NOP',
                    'SEA': 'OKC'}

    df['TEAM_ABBREVIATION'] = df['TEAM_ABBREVIATION'].replace(abbr_mapping)
    df['MATCHUP'] = df['MATCHUP'].str.replace('NJN', 'BKN')
    df['MATCHUP'] = df['MATCHUP'].str.replace('CHH', 'CHA')
    df['MATCHUP'] = df['MATCHUP'].str.replace('VAN', 'MEM')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOH', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('NOK', 'NOP')
    df['MATCHUP'] = df['MATCHUP'].str.replace('SEA', 'OKC')

    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

    df['HOME_GAME'] = df['MATCHUP'].str.contains('vs').astype(int)

    return df


# from src.data.make_team_dataset import prep_for_aggregation

def prep_for_aggregation(df):
    """This function...
    1) Removes categories that are percentages,
    as we will be averaging them and do not want to average 
    percentages. 
    2) Converts shooting percentage stats into raw values"""
    df = df.copy()

    df = df.drop(columns=['FT_PCT', 'FG_PCT', 'FG3_PCT', 'DREB_PCT',
                          'OREB_PCT', 'REB_PCT', 'AST_PCT', 'AST_TOV',
                          'AST_RATIO', 'E_TM_TOV_PCT', 'TM_TOV_PCT',
                          'EFG_PCT', 'TS_PCT', 'USG_PCT', 'E_USG_PCT',
                          'PACE_PER40', 'MIN', 'PIE', 'CFG_PCT', 'UFG_PCT',
                          'DFG_PCT', 'E_OFF_RATING', 'E_DEF_RATING', 'E_NET_RATING'])

    df['FG2M'] = df['FGM'] - df['FG3M']
    df['FG2A'] = df['FGA'] - df['FG3A']
    df['PTS_2PT_MR'] = (df['PTS'] * df['PCT_PTS_2PT_MR']).astype('int8')
    df['PTS_FB'] = (df['PTS'] * df['PCT_PTS_FB']).astype('int8')
    df['PTS_OFF_TOV'] = (df['PTS'] * df['PCT_PTS_OFF_TOV']).astype('int8')
    df['PTS_PAINT'] = (df['PTS'] * df['PCT_PTS_PAINT']).astype('int8')
    df['AST_2PM'] = (df['FG2M'] * df['PCT_AST_2PM']).astype('int8')
    df['AST_3PM'] = (df['FG3M'] * df['PCT_AST_3PM']).astype('int8')
    df['UAST_2PM'] = (df['FG2M'] * df['PCT_UAST_2PM']).astype('int8')
    df['UAST_3PM'] = (df['FG3M'] * df['PCT_UAST_3PM']).astype('int8')

    df['POINT_DIFF'] = df['PLUS_MINUS']
    df['RECORD'] = df['WL']
    df['TEAM_SCORE'] = df['PTS']
    
    df = df.drop(columns = ['PCT_FGA_2PT', 'PCT_FGA_3PT', 'PCT_PTS_2PT',
                          'PCT_PTS_2PT_MR', 'PCT_PTS_3PT', 'PCT_PTS_FB',
                          'PCT_PTS_FT','PCT_PTS_OFF_TOV', 'PCT_PTS_PAINT', 
                          'PCT_AST_2PM', 'PCT_UAST_2PM','PCT_AST_3PM',
                          'PCT_UAST_3PM', 'PCT_AST_FGM', 'PCT_UAST_FGM',
                          'E_PACE'])
    
    ## Reorder Columns
    
    df = df[['SEASON', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'TEAM_SCORE', 'WL', 'POINT_DIFF', 'HOME_GAME', 'RECORD',
       'FG2M', 'FG2A', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB', 
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PLUS_MINUS', 'OFF_RATING', 'DEF_RATING', 'NET_RATING', 'PACE',
       'POSS', 'DIST', 'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS',
       'CFGM', 'CFGA', 'UFGM', 'UFGA', 'DFGM', 'DFGA', 'PTS_2PT_MR', 'PTS_FB', 'PTS_OFF_TOV', 'PTS_PAINT', 'AST_2PM',
       'AST_3PM', 'UAST_2PM', 'UAST_3PM']]

    return df


def load_betting_data(conn):
    spreads = pd.read_sql("SELECT * FROM spreads", conn)
    moneylines = pd.read_sql("SELECT * FROM moneylines", conn)

    return spreads, moneylines


def convert_american_to_decimal(x):
    return np.where(x>0, (100+x)/100, 1+(100.0/-x))          


def clean_moneyline_df(df):
    abbr_mapping = {'Boston': 'BOS', 'Portland': 'POR',
                    'L.A. Lakers': 'LAL', 'Brooklyn': 'BKN',
                    'Cleveland': 'CLE', 'Toronto': 'TOR',
                    'Philadelphia': 'PHI', 'Memphis': 'MEM',
                    'Minnesota': 'MIN', 'New Orleans': 'NOP',
                    'Oklahoma City': 'OKC', 'Dallas': 'DAL',
                    'San Antonio': 'SAS', 'Denver': 'DEN',
                    'Golden State': 'GSW', 'L.A. Clippers': 'LAC',
                    'Orlando': 'ORL', 'Utah': 'UTA',
                    'Charlotte': 'CHA', 'Detroit': 'DET',
                    'Miami': 'MIA', 'Phoenix': 'PHX',
                    'Atlanta': 'ATL', 'New York': 'NYK',
                    'Indiana': 'IND', 'Chicago': 'CHI',
                    'Houston': 'HOU', 'Milwaukee': 'MIL',
                    'Sacramento': 'SAC', 'Washington': 'WAS'}

    df['HOME_TEAM'] = df['HOME_TEAM'].replace(abbr_mapping)
    df['AWAY_TEAM'] = df['AWAY_TEAM'].replace(abbr_mapping)

    away_mls = df['AWAY_ML'].str.split(",", expand=True)
    home_mls = df['HOME_ML'].str.split(",", expand=True)

    away_mls = away_mls.replace('-', np.nan).replace('', np.nan)
    away_mls = away_mls.fillna(value=np.nan)
    away_mls = away_mls.astype(float)

    home_mls = home_mls.replace('-', np.nan).replace('', np.nan)
    home_mls = home_mls.fillna(value=np.nan)
    home_mls = home_mls.astype(float)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
    
        highest_away_ml = away_mls.apply(lambda row: np.nanmax(
            abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
        highest_away_ml = convert_american_to_decimal(highest_away_ml)
        highest_away_ml = pd.DataFrame(
            highest_away_ml, columns=['HIGHEST_AWAY_ML'])

        highest_home_ml = home_mls.apply(lambda row: np.nanmax(
            abs(row)) if np.nanmax(row) > 0 else -np.nanmax(abs(row)), axis=1)
        highest_home_ml = convert_american_to_decimal(highest_home_ml)
        highest_home_ml = pd.DataFrame(
            highest_home_ml, columns=['HIGHEST_HOME_ML'])

    moneylines = pd.concat(
        [df.iloc[:, :4], highest_home_ml, highest_away_ml], axis=1)
    
    moneylines['GM_DATE'] = pd.to_datetime(moneylines['GM_DATE'])

    return moneylines

def clean_spreads_df(df):
    abbr_mapping = {'Boston': 'BOS', 'Portland': 'POR',
                    'L.A. Lakers': 'LAL', 'Brooklyn': 'BKN',
                    'Cleveland': 'CLE', 'Toronto': 'TOR',
                    'Philadelphia': 'PHI', 'Memphis': 'MEM',
                    'Minnesota': 'MIN', 'New Orleans': 'NOP',
                    'Oklahoma City': 'OKC', 'Dallas': 'DAL',
                    'San Antonio': 'SAS', 'Denver': 'DEN',
                    'Golden State': 'GSW', 'L.A. Clippers': 'LAC',
                    'Orlando': 'ORL', 'Utah': 'UTA',
                    'Charlotte': 'CHA', 'Detroit': 'DET',
                    'Miami': 'MIA', 'Phoenix': 'PHX',
                    'Atlanta': 'ATL', 'New York': 'NYK',
                    'Indiana': 'IND', 'Chicago': 'CHI',
                    'Houston': 'HOU', 'Milwaukee': 'MIL',
                    'Sacramento': 'SAC', 'Washington': 'WAS'}

    df['HOME_TEAM'] = df['HOME_TEAM'].replace(abbr_mapping)
    df['AWAY_TEAM'] = df['AWAY_TEAM'].replace(abbr_mapping)

    away_spreads = df['AWAY_SPREAD'].str.split(",", expand=True)
    home_spreads = df['HOME_SPREAD'].str.split(",", expand=True)

    for col in away_spreads.columns:
        away_spreads[col] = away_spreads[col].str[:-4]
        away_spreads[col] = away_spreads[col].str.replace('½', '.5')
        away_spreads[col] = away_spreads[col].str.replace('PK', '0')

        away_spreads[col] = away_spreads[col].astype(str).apply(
            lambda x: x if x == '' else (x[:-1] if x[-1] == '-' else x))

    away_spreads = away_spreads.replace('-', np.nan)
    away_spreads = away_spreads.replace('', np.nan)
    away_spreads = away_spreads.replace('None', np.nan)
    away_spreads = away_spreads.fillna(value=np.nan)

    away_spreads = away_spreads.astype(float)

    for col in home_spreads.columns:
        home_spreads[col] = home_spreads[col].str[:-4]
        home_spreads[col] = home_spreads[col].str.replace('½', '.5')
        home_spreads[col] = home_spreads[col].str.replace('PK', '0')

        home_spreads[col] = home_spreads[col].astype(str).apply(
            lambda x: x if x == '' else (x[:-1] if x[-1] == '-' else x))

    home_spreads = home_spreads.replace('-', np.nan).replace('', np.nan).replace('None', np.nan)
    home_spreads = home_spreads.fillna(value=np.nan)

    home_spreads = home_spreads.astype(float)

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)

        highest_away_spread = away_spreads.apply(
            lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
        highest_away_spread = pd.DataFrame(
            highest_away_spread, columns=['HIGHEST_AWAY_SPREAD'])

        highest_home_spread = home_spreads.apply(
            lambda row: -np.nanmax(abs(row)) if np.nanmax(row) < 0 else np.nanmax(abs(row)), axis=1)
        highest_home_spread = pd.DataFrame(
            highest_home_spread, columns=['HIGHEST_HOME_SPREAD'])

    spreads = pd.concat(
        [df.iloc[:, :4], highest_home_spread, highest_away_spread], axis=1)
    spreads['GM_DATE'] = pd.to_datetime(spreads['GM_DATE'])

    return spreads


def merge_betting_and_boxscore_data(clean_spreads, clean_mls, clean_boxscores):
    clean_boxscores['HOME_TEAM'] = clean_boxscores['MATCHUP'].apply(
        lambda x: x[:3] if 'vs' in x else x[-3:])
    clean_boxscores['AWAY_TEAM'] = clean_boxscores['MATCHUP'].apply(
        lambda x: x[:3] if '@' in x else x[-3:])

    temp = pd.merge(clean_mls, clean_spreads, on=[
                    'SEASON', 'GM_DATE', 'HOME_TEAM', 'AWAY_TEAM'])

    merged_df = pd.merge(clean_boxscores, temp, how='left', 
                         left_on=['SEASON', 'HOME_TEAM', 'AWAY_TEAM', 'GAME_DATE'],
                         right_on=['SEASON', 'HOME_TEAM', 'AWAY_TEAM', 'GM_DATE'])

    merged_df['ML'] = merged_df.apply(lambda row: row['HIGHEST_HOME_ML'] if row['HOME_GAME'] == 1
                                      else row['HIGHEST_AWAY_ML'], axis=1)

    merged_df['SPREAD'] = merged_df.apply(lambda row: row['HIGHEST_HOME_SPREAD'] if row['HOME_GAME'] == 1
                                          else -row['HIGHEST_HOME_SPREAD'], axis=1)

    merged_df = merged_df.drop(columns=['HOME_TEAM', 'AWAY_TEAM', 'GM_DATE',
                                        'HIGHEST_HOME_ML', 'HIGHEST_AWAY_ML',
                                        'HIGHEST_HOME_SPREAD', 'HIGHEST_AWAY_SPREAD'])

    merged_df['ATS_DIFF'] = merged_df['POINT_DIFF'] + merged_df['SPREAD']

    merged_df['TEAM_COVERED'] = (merged_df['ATS_DIFF'] > 0).astype(int)
    

    return merged_df


def normalize_per_100_poss(df):
    df = df.copy(deep=True)
    
    df.iloc[:, 12:27] = 100*df.iloc[:, 12:27].div(df['PACE'], axis=0) 
    df.iloc[:,  34:-4] = 100*df.iloc[:, 34:-4].div(df['PACE'], axis=0) 
    
    return df


def create_matchups(df):
    """This function makes each row a matchup between 
    team and opp"""
    df = df.copy()
    

    matchups = pd.merge(df, df.iloc[:, :-4], on=['GAME_ID'], suffixes=['', '_opp'])
    matchups = matchups.loc[matchups['TEAM_ABBREVIATION'] != matchups['TEAM_ABBREVIATION_opp']]

    matchups = matchups.drop(columns = ['SEASON_opp', 'TEAM_ABBREVIATION_opp', 'GAME_DATE_opp',
                                        'MATCHUP_opp', 'HOME_GAME_opp', 'TEAM_NAME_opp', 
                                        'TEAM_ID_opp', 'WL_opp']
                             )
    
    matchups
    
    return matchups


def build_team_avg_stats_df(df: pd.DataFrame, span = 10) -> pd.DataFrame:    
    """This function finds the average for each team and opp statistic up to (and NOT including) the given date.
    """
    
    df = df.copy(deep=True)

    df = df.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE']).reset_index(drop=True)

    

    drop_cols = ['TEAM_ID', 'TEAM_NAME', 'GAME_ID', 'MATCHUP', 
                 'HOME_GAME', 'TEAM_SCORE', 'ML', 'SPREAD', 
                'GAME_DATE', 'POINT_DIFF', 'WL', 'TEAM_SCORE_opp',
                'POINT_DIFF_opp', 'RECORD', 'RECORD_opp', 'TEAM_COVERED']

    stats = df.drop(columns=drop_cols)

    avg_stat_holder = []

    for stat in stats.columns[2:]:
        avg_stats = stats.groupby(['TEAM_ABBREVIATION'])[stat].ewm(span=span).mean().reset_index(drop=True)
        avg_stat_holder.append(avg_stats)
    
    
    matchup_info = df[['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                          'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                          'ML', 'SPREAD', 'ATS_DIFF', 'RECORD', 'TEAM_COVERED', 
                          'POINT_DIFF', 'WL']]   

    avg_stats = pd.concat(avg_stat_holder, axis=1)
    

    avg_stats = avg_stats.rename(columns={'ATS_DIFF':'AVG_ATS_DIFF'})
    
    avg_stats = pd.concat([matchup_info, avg_stats], axis=1)
    
    avg_stats['WIN_PCT'] = avg_stats.groupby(['TEAM_ABBREVIATION'])['RECORD'].rolling(window=span).mean().values
    avg_stats['COVER_PCT'] = avg_stats.groupby(['TEAM_ABBREVIATION'])['TEAM_COVERED'].rolling(window=span).mean().values

    avg_stats = avg_stats.drop(columns='RECORD')

    avg_stats = avg_stats.sort_values(['TEAM_ABBREVIATION', 'GAME_DATE'])
    # avg_stats.iloc[:, 14:] = avg_stats.iloc[:, 14:].shift(1).where(avg_stats['TEAM_ABBREVIATION'].eq(avg_stats['TEAM_ABBREVIATION'].shift()))

    avg_stats = avg_stats.add_suffix('_L{}'.format(span))
    
    avg_stats = avg_stats.rename(columns = {'SEASON_L{}'.format(span):'SEASON',
                                           'TEAM_ABBREVIATION_L{}'.format(span):'TEAM_ABBREVIATION',
                                           'GAME_DATE_L{}'.format(span):'GAME_DATE',
                                           'GAME_ID_L{}'.format(span):'GAME_ID',
                                           'MATCHUP_L{}'.format(span): 'MATCHUP', 
                                           'HOME_GAME_L{}'.format(span): 'HOME_GAME', 
                                           'TEAM_SCORE_L{}'.format(span):'TEAM_SCORE',
                                           'ML_L{}'.format(span):'ML', 
                                           'SPREAD_L{}'.format(span):'SPREAD',
                                           'ATS_DIFF_L{}'.format(span):'ATS_DIFF',
                                           'RECORD_L{}'.format(span):'RECORD', 
                                           'TEAM_COVERED_L{}'.format(span):'TEAM_COVERED',
                                           'POINT_DIFF_L{}'.format(span):'POINT_DIFF',
                                           'WL_L{}'.format(span):'WL'})
    
    return avg_stats


def add_percentage_features(df, span):
    """Add the following features for both team and opp:
    OREB_PCT, DREB_PCT, REB_PCT, TS_PCT, EFG_PCT, AST_RATIO, TOV_PCT, PIE.
    """
    
    df = df.copy()
    
    df['OREB_PCT_L{}'.format(span)] = df['OREB_L{}'.format(span)] / (df['OREB_L{}'.format(span)] + df['DREB_opp_L{}'.format(span)])
    df['OREB_PCT_opp_L{}'.format(span)] = df['OREB_opp_L{}'.format(span)] / (df['OREB_opp_L{}'.format(span)] + df['DREB_L{}'.format(span)])

    df['DREB_PCT_L{}'.format(span)] = df['DREB_L{}'.format(span)] / (df['DREB_L{}'.format(span)] + df['OREB_opp_L{}'.format(span)])
    df['DREB_PCT_opp_L{}'.format(span)] = df['DREB_opp_L{}'.format(span)] / (df['DREB_opp_L{}'.format(span)] + df['OREB_L{}'.format(span)])

    df['REB_PCT_L{}'.format(span)] = df['REB_L{}'.format(span)] / (df['REB_L{}'.format(span)] + df['REB_opp_L{}'.format(span)])
    df['REB_PCT_opp_L{}'.format(span)] = df['REB_opp_L{}'.format(span)] / (df['REB_opp_L{}'.format(span)] + df['REB_L{}'.format(span)])

    df['TS_PCT_L{}'.format(span)] = df['PTS_L{}'.format(span)] / ((2*(df['FG2A_L{}'.format(span)] + df['FG3A_L{}'.format(span)]) + 0.44*df['FTA_L{}'.format(span)]))
    
    df['TS_PCT_opp_L{}'.format(span)] = df['PTS_opp_L{}'.format(span)] / ((2*(df['FG2A_opp_L{}'.format(span)] + df['FG3A_opp_L{}'.format(span)]) + 0.44*df['FTA_opp_L{}'.format(span)]))

    df['EFG_PCT_L{}'.format(span)] = (df['FG2M_L{}'.format(span)] + 1.5*df['FG3M_L{}'.format(span)]) / (df['FG2A_L{}'.format(span)]
                                                                    + df['FG3A_L{}'.format(span)])
    df['EFG_PCT_opp_L{}'.format(span)] = (df['FG2M_opp_L{}'.format(span)] + 1.5*df['FG3M_opp_L{}'.format(span)]) / (df['FG2A_opp_L{}'.format(span)] 
                                                                 + df['FG3A_opp_L{}'.format(span)])

    df['AST_RATIO_L{}'.format(span)] = (df['AST_L{}'.format(span)] * 100) / df['PACE_L{}'.format(span)]
    df['AST_RATIO_opp_L{}'.format(span)] = (df['AST_opp_L{}'.format(span)] * 100) / df['PACE_opp_L{}'.format(span)]

    df['TOV_PCT_L{}'.format(span)] = 100*df['TOV_L{}'.format(span)] / (df['FG2A_L{}'.format(span)] 
                                               + df['FG3A_L{}'.format(span)] 
                                               + 0.44*df['FTA_L{}'.format(span)] 
                                               + df['TOV_L{}'.format(span)])
    
    df['TOV_PCT_opp_L{}'.format(span)] = 100*df['TOV_opp_L{}'.format(span)] / (df['FG2A_opp_L{}'.format(span)] 
                                             + df['FG3A_opp_L{}'.format(span)] 
                                             + 0.44*df['FTA_opp_L{}'.format(span)] 
                                             + df['TOV_opp_L{}'.format(span)])
    
    
    df['PIE_L{}'.format(span)] = ((df['PTS_L{}'.format(span)] + df['FG2M_L{}'.format(span)] + df['FG3M_L{}'.format(span)] + df['FTM_L{}'.format(span)] 
                 - df['FG2A_L{}'.format(span)] - df['FG3A_L{}'.format(span)] - df['FTA_L{}'.format(span)] 
                 + df['DREB_L{}'.format(span)] + df['OREB_L{}'.format(span)]/2
                + df['AST_L{}'.format(span)] + df['STL_L{}'.format(span)] + df['BLK_L{}'.format(span)]/2
                - df['PF_L{}'.format(span)] - df['TOV_L{}'.format(span)]) 
                 / (df['PTS_L{}'.format(span)] + df['PTS_opp_L{}'.format(span)] + df['FG2M_L{}'.format(span)] + df['FG2M_opp_L{}'.format(span)]
                   + df['FG3M_L{}'.format(span)] + df['FG3M_opp_L{}'.format(span)] + df['FTM_L{}'.format(span)] + df['FTM_opp_L{}'.format(span)]
                   - df['FG2A_L{}'.format(span)] - df['FG2A_opp_L{}'.format(span)] - df['FG3A_L{}'.format(span)] - df['FG3A_opp_L{}'.format(span)] 
                    - df['FTA_L{}'.format(span)] - df['FTA_opp_L{}'.format(span)] + df['DREB_L{}'.format(span)] + df['DREB_opp_L{}'.format(span)]
                    + (df['OREB_L{}'.format(span)]+df['OREB_opp_L{}'.format(span)])/2 + df['AST_L{}'.format(span)] + df['AST_opp_L{}'.format(span)]
                    + df['STL_L{}'.format(span)] + df['STL_opp_L{}'.format(span)] + (df['BLK_L{}'.format(span)] + df['BLK_opp_L{}'.format(span)])/2
                    - df['PF_L{}'.format(span)] - df['PF_opp_L{}'.format(span)] - df['TOV_L{}'.format(span)] - df['TOV_opp_L{}'.format(span)]))
        
    return df


def add_rest_days_for_model(df):
    df['REST'] = np.nan
    for team in df['TEAM_ABBREVIATION'].unique():
        team_df = df.loc[df['TEAM_ABBREVIATION'] == team].sort_values('GAME_DATE')
        idx = team_df.index
        team_df['REST'] = (team_df['GAME_DATE'].shift(-1) - team_df['GAME_DATE']) / np.timedelta64(1, 'D')
        team_df.at[max(idx), 'REST'] = (pd.to_datetime(date.today()) - team_df.at[max(idx), 'GAME_DATE']) / np.timedelta64(1, 'D')

        df.loc[idx, 'REST'] = team_df['REST']
        df.loc[df['REST'] >= 8, 'REST'] = 8
            
    return df


def season_to_string(x):
    return str(x) + '-' + str(x+1)[-2:]


def load_and_process_data(start_season, end_season):
    start_season = season_to_string(start_season)
    end_season = season_to_string(end_season)

    db_filepath = Path.home().joinpath('NBA_model_v1', 'data', 'nba.db')

    conn = sqlite3.connect(db_filepath)
    
    print("Loading raw team boxscore data from sql database...")
    
    df = load_team_data(conn, start_season, end_season)
    print("Loading betting data from sql database...")
    spreads, moneylines = load_betting_data(conn)
    
    print("Cleaning Data...")
    df = clean_team_data(df)
    df = prep_for_aggregation(df)

    clean_mls = clean_moneyline_df(df = moneylines)
    clean_spreads = clean_spreads_df(df = spreads)
    
    print("Merging Boxscore and Betting Data...")
    merged_df = merge_betting_and_boxscore_data(
        clean_spreads, clean_mls, clean_boxscores = df)
    
    
    stats_per_100 = normalize_per_100_poss(merged_df)

    print("Aggregating over last 5, 10, and 20 game windows")
    
    matchups = create_matchups(stats_per_100)
    
    team_stats_ewa_5 = build_team_avg_stats_df(matchups, span=5)
    team_stats_ewa_5 = add_percentage_features(team_stats_ewa_5, span=5)

    team_stats_ewa_10 = build_team_avg_stats_df(matchups, span=10)
    team_stats_ewa_10 = add_percentage_features(team_stats_ewa_10, span=10)

    team_stats_ewa_20 = build_team_avg_stats_df(matchups, span=20)
    team_stats_ewa_20 = add_percentage_features(team_stats_ewa_20, span=20)


    temp = pd.merge(team_stats_ewa_5, team_stats_ewa_10, how='inner',
                    on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                        'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                        'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                        'POINT_DIFF', 'WL'])

    df_full = pd.merge(temp, team_stats_ewa_20, how='inner', 
                       on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                            'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                            'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                            'POINT_DIFF', 'WL'])

    df_full = df_full.sort_values(['GAME_DATE', 'GAME_ID', 'HOME_GAME'])
    
    
    columns_to_drop = ['PTS_L5', 'PTS_L10', 'PTS_L20',
                        'PLUS_MINUS_L5', 'PLUS_MINUS_L10', 'PLUS_MINUS_L20',
                        'NET_RATING_L5', 'NET_RATING_L10', 'NET_RATING_L20',
                        'POSS_L5', 'POSS_L10', 'POSS_L20',
                        'REB_L5', 'REB_L10', 'REB_L20',
                        'REB_opp_L5', 'REB_opp_L10', 'REB_opp_L20',
                        'PTS_opp_L5', 'PTS_opp_L10', 'PTS_opp_L20',
                        'PLUS_MINUS_opp_L5', 'PLUS_MINUS_opp_L10', 'PLUS_MINUS_opp_L20',
                        'NET_RATING_opp_L5', 'NET_RATING_opp_L10', 'NET_RATING_opp_L20',
                        'POSS_opp_L5', 'POSS_opp_L10', 'POSS_opp_L20']
    
    df_full = df_full.drop(columns = columns_to_drop)
    
    print("adding rest days")
    df_full = add_rest_days_for_model(df_full)
    
    return df_full
    

def make_matchup_row(home_team, away_team, df):
    
    print("creating matchups between Home and Away team aggregated stats")

    matchup_info_cols = ['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE', 'GAME_ID', 'MATCHUP',
        'HOME_GAME', 'TEAM_SCORE', 'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED',
        'POINT_DIFF', 'WL']

    most_recent_home_stats = df.loc[df['TEAM_ABBREVIATION'] == home_team].tail(1).drop(columns=matchup_info_cols).values
    most_recent_away_stats = df.loc[df['TEAM_ABBREVIATION'] == away_team].tail(1).drop(columns=matchup_info_cols).values

    matchup_row = pd.DataFrame(np.concatenate([most_recent_home_stats, most_recent_away_stats], axis=1), columns=X_train.columns)
        
    return matchup_row

In [328]:
def add_rest_days_adv(df):
    
    df['prev_game'] = df.groupby(['SEASON', 'TEAM_ABBREVIATION'])['GAME_DATE'].shift(1)

    df['REST'] = (df['GAME_DATE'] - df['prev_game']) / np.timedelta64(1, 'D')
            
    df.loc[df['REST'] >= 7, 'REST'] = 7
    
    df['3daysago'] = df['GAME_DATE'] - timedelta(days=3)
    df['2daysago'] = df['GAME_DATE'] - timedelta(days=2)
    df['1daysago'] = df['GAME_DATE'] - timedelta(days=1)

    df = pd.merge(df, df[['TEAM_ABBREVIATION', 'GAME_DATE']], 
                    how='left',
                    left_on = ['TEAM_ABBREVIATION', '1daysago'],
                    right_on = ['TEAM_ABBREVIATION', 'GAME_DATE'],
                    suffixes=['', '_1'])

    df = pd.merge(df, df[['TEAM_ABBREVIATION', 'GAME_DATE']], 
                    how='left',
                    left_on = ['TEAM_ABBREVIATION', '2daysago'],
                    right_on = ['TEAM_ABBREVIATION', 'GAME_DATE'],
                    suffixes=['', '_2'])

    df = pd.merge(df, df[['TEAM_ABBREVIATION', 'GAME_DATE']], 
                    how='left',
                    left_on = ['TEAM_ABBREVIATION', '3daysago'],
                    right_on = ['TEAM_ABBREVIATION', 'GAME_DATE'],
                    suffixes=['', '_3'])

    df[['GAME_DATE_1', 'GAME_DATE_2', 'GAME_DATE_3']] = df[['GAME_DATE_1', 'GAME_DATE_2', 'GAME_DATE_3']].notnull().astype(int)

    df['Threein4B2B'] = ((df['GAME_DATE_1'] == 1) & (df['GAME_DATE_2'] == 0) & (df['GAME_DATE_3'] == 1)).astype(int)
    df['Threein4'] = ((df['GAME_DATE_1'] == 0) & (df['GAME_DATE_2'] == 1) & (df['GAME_DATE_3'] == 1)).astype(int)
    
    df = df.drop(columns=['prev_game', '3daysago', '2daysago', '1daysago', 'GAME_DATE_1', 'GAME_DATE_2', 'GAME_DATE_3'])
    
    return df

In [320]:
connection = sqlite3.connect(db_filepath)
start_season = 2013
end_season = 2022

df = load_team_data(conn=connection, start_season=start_season, end_season=end_season)

In [343]:
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
todays_matchups = pd.DataFrame({'SEASON':['2022-23', '2022-23'],
                                'TEAM_ABBREVIATION':['CLE', 'DEN'],
                                'GAME_DATE':['2022-10-25', '2022-10-25']})

todays_matchups['GAME_DATE'] = pd.to_datetime(todays_matchups['GAME_DATE'])
df_new = pd.concat([df, todays_matchups])

In [344]:
add_rest_days_adv(df_new)

,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,E_NET_RATING,NET_RATING,AST_PCT,AST_TOV,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,E_TM_TOV_PCT,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE,PCT_FGA_2PT,PCT_FGA_3PT,PCT_PTS_2PT,PCT_PTS_2PT_MR,PCT_PTS_3PT,PCT_PTS_FB,PCT_PTS_FT,PCT_PTS_OFF_TOV,PCT_PTS_PAINT,PCT_AST_2PM,PCT_UAST_2PM,PCT_AST_3PM,PCT_UAST_3PM,PCT_AST_FGM,PCT_UAST_FGM,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,CFG_PCT,UFGM,UFGA,UFG_PCT,DFGM,DFGA,DFG_PCT,REST,Threein4B2B,Threein4
0,2013-14,1610612746,LAC,Los Angeles Clippers,0021300003,2013-10-29,LAC @ LAL,L,240.0,41.0,83.0,0.494,8.0,21.0,0.381,13.0,23.0,0.565,10.0,30.0,40.0,27.0,11.0,4.0,16.0,21.0,103.0,-13.0,103.9,105.1,109.1,117.2,-5.2,-12.1,0.659,1.69,19.8,0.255,0.561,0.423,16.142,16.3,0.542,0.553,1.0,0.200,102.72,98.5,82.08,98.0,0.497,0.747,0.253,0.641,0.175,0.233,0.184,0.126,0.243,0.466,0.606,0.394,0.875,0.125,0.659,0.341,17.44,40.0,64.0,98.0,394.0,0.0,2.0,279.0,23.0,44.0,0.523,18.0,39.0,0.462,12.0,21.0,0.571,NaN,0,0
1,2013-14,1610612747,LAL,Los Angeles Lakers,0021300003,2013-10-29,LAL vs. LAC,W,240.0,42.0,93.0,0.452,14.0,29.0,0.483,18.0,28.0,0.643,18.0,34.0,52.0,23.0,8.0,6.0,19.0,23.0,116.0,13.0,109.1,117.2,103.9,105.1,5.2,12.1,0.548,1.21,15.6,0.439,0.745,0.577,17.871,19.2,0.527,0.551,1.0,0.195,102.72,98.5,82.08,99.0,0.503,0.688,0.312,0.483,0.121,0.362,0.112,0.155,0.241,0.362,0.429,0.571,0.786,0.214,0.548,0.452,17.35,44.0,64.0,103.0,449.0,4.0,0.0,318.0,15.0,31.0,0.484,27.0,62.0,0.435,20.0,28.0,0.714,NaN,0,0
2,2013-14,1610612741,CHI,Chicago Bulls,0021300002,2013-10-29,CHI @ MIA,L,240.0,35.0,83.0,0.422,7.0,26.0,0.269,18.0,23.0,0.783,11.0,30.0,41.0,23.0,11.0,4.0,19.0,27.0,95.0,-12.0,93.9,96.9,107.3,108.1,-13.3,-11.1,0.657,1.21,17.0,0.294,NaN,NaN,18.790,19.4,0.464,0.510,1.0,0.198,100.44,98.5,82.08,98.0,0.401,0.687,0.313,0.589,0.063,0.221,0.179,0.189,0.316,0.526,0.571,0.429,1.000,0.000,0.657,0.343,17.64,37.0,57.0,87.0,446.0,3.0,3.0,317.0,17.0,35.0,0.486,18.0,48.0,0.375,11.0,20.0,0.550,NaN,0,0
3,2013-14,1610612748,MIA,Miami Heat,0021300002,2013-10-29,MIA vs. CHI,W,240.0,37.0,72.0,0.514,11.0,20.0,0.550,22.0,29.0,0.759,5.0,35.0,40.0,26.0,10.0,7.0,20.0,21.0,107.0,12.0,107.3,108.1,93.9,96.9,13.3,11.1,0.703,1.30,19.9,NaN,0.706,NaN,20.048,20.2,0.590,0.631,1.0,0.197,100.44,98.5,82.08,99.0,0.599,0.722,0.278,0.486,0.168,0.308,0.093,0.206,0.140,0.318,0.577,0.423,1.000,0.000,0.703,0.297,17.01,22.0,73.0,93.0,462.0,3.0,3.0,340.0,11.0,28.0,0.393,26.0,44.0,0.591,17.0,26.0,0.654,NaN,0,0
4,2013-14,1610612754,IND,Indiana Pacers,0021300001,2013-10-29,IND vs. ORL,W,240.0,34.0,71.0,0.479,7.0,17.0,0.412,22.0,32.0,0.688,10.0,34.0,44.0,17.0,4.0,18.0,21.0,13.0,97.0,10.0,101.0,103.2,84.1,92.6,16.8,10.6,0.500,0.81,13.8,0.317,0.617,0.495,21.857,22.3,0.528,0.570,1.0,0.198,99.74,94.0,78.33,94.0,0.661,0.761,0.239,0.557,0.186,0.216,0.134,0.227,0.299,0.371,0.444,0.556,0.714,0.286,0.500,0.500,16.09,40.0,78.0,117.0,428.0,1.0,1.0,310.0,20.0,38.0,0.526,14.0,33.0,0.424,9.0,21.0,0.429,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23105,2022-23,1610612745,HOU,Houston Rockets,0022200048,2022-10-24,HOU vs. UTA,W,240.0,38.0,90.0,0.422,11.0,35.0,0.314,27.0,32.0,0.844,17.0,30.0,47.0,17.0,7.0,7.0,16.0,20.0,114.0,6.0,110.6,114.0,105.8,109.1,4.8,4.9,0.447,1.06,12.4,0.418,0.593,0.505,15.522,16.0,0.483,0.548,1.0,0.198,102.56,99.5,82.92,100.0,0.518,0.611,0.389,0.474,0.018,0.289,0.053,0.237,0.167,0.456,0.370,0.630,0.636,0.364,0.447,0.553,17.47,37.0,56.0,89.0,363.0,3.0,2.0,232.0,15.0,43.0,0.349,23.0,47.0,0.489,25.0,3

In [295]:
df_full = load_and_process_data(2013, 2022)

Loading raw team boxscore data from sql database...
Loading betting data from sql database...
Cleaning Data...
Merging Boxscore and Betting Data...
Aggregating over last 5, 10, and 20 game windows
adding rest days


In [132]:
df_full

,SEASON,TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_GAME,TEAM_SCORE,ML,SPREAD,ATS_DIFF,TEAM_COVERED,POINT_DIFF,WL,FG2M_L5,FG2A_L5,FG3M_L5,FG3A_L5,FTM_L5,FTA_L5,OREB_L5,DREB_L5,AST_L5,STL_L5,BLK_L5,TOV_L5,PF_L5,OFF_RATING_L5,DEF_RATING_L5,PACE_L5,DIST_L5,ORBC_L5,DRBC_L5,RBC_L5,TCHS_L5,SAST_L5,FTAST_L5,PASS_L5,CFGM_L5,CFGA_L5,UFGM_L5,UFGA_L5,DFGM_L5,DFGA_L5,PTS_2PT_MR_L5,PTS_FB_L5,PTS_OFF_TOV_L5,PTS_PAINT_L5,AST_2PM_L5,AST_3PM_L5,UAST_2PM_L5,UAST_3PM_L5,AVG_ATS_DIFF_L5,FG2M_opp_L5,FG2A_opp_L5,FG3M_opp_L5,FG3A_opp_L5,FTM_opp_L5,FTA_opp_L5,OREB_opp_L5,DREB_opp_L5,AST_opp_L5,STL_opp_L5,BLK_opp_L5,TOV_opp_L5,PF_opp_L5,OFF_RATING_opp_L5,DEF_RATING_opp_L5,PACE_opp_L5,DIST_opp_L5,ORBC_opp_L5,DRBC_opp_L5,RBC_opp_L5,TCHS_opp_L5,SAST_opp_L5,FTAST_opp_L5,PASS_opp_L5,CFGM_opp_L5,CFGA_opp_L5,UFGM_opp_L5,UFGA_opp_L5,DFGM_opp_L5,DFGA_opp_L5,PTS_2PT_MR_opp_L5,PTS_FB_opp_L5,PTS_OFF_TOV_opp_L5,PTS_PAINT_opp_L5,AST_2PM_opp_L5,AST_3PM_opp_L5,UAST_2PM_opp_L5,UAST_3PM_opp_L5,WIN_PCT_L5,COVER_PCT_L5,OREB_PCT_L5,OREB_PCT_opp_L5,DREB_PCT_L5,DREB_PCT_opp_L5,REB_PCT_L5,REB_PCT_opp_L5,TS_PCT_L5,TS_PCT_opp_L5,EFG_PCT_L5,EFG_PCT_opp_L5,AST_RATIO_L5,AST_RATIO_opp_L5,TOV_PCT_L5,TOV_PCT_opp_L5,PIE_L5,FG2M_L10,FG2A_L10,FG3M_L10,FG3A_L10,FTM_L10,FTA_L10,OREB_L10,DREB_L10,AST_L10,STL_L10,BLK_L10,TOV_L10,PF_L10,OFF_RATING_L10,DEF_RATING_L10,PACE_L10,DIST_L10,ORBC_L10,DRBC_L10,RBC_L10,TCHS_L10,SAST_L10,FTAST_L10,PASS_L10,CFGM_L10,CFGA_L10,UFGM_L10,UFGA_L10,DFGM_L10,DFGA_L10,PTS_2PT_MR_L10,PTS_FB_L10,PTS_OFF_TOV_L10,PTS_PAINT_L10,AST_2PM_L10,AST_3PM_L10,UAST_2PM_L10,UAST_3PM_L10,AVG_ATS_DIFF_L10,FG2M_opp_L10,FG2A_opp_L10,FG3M_opp_L10,FG3A_opp_L10,FTM_opp_L10,FTA_opp_L10,OREB_opp_L10,DREB_opp_L10,AST_opp_L10,STL_opp_L10,BLK_opp_L10,TOV_opp_L10,PF_opp_L10,OFF_RATING_opp_L10,DEF_RATING_opp_L10,PACE_opp_L10,DIST_opp_L10,ORBC_opp_L10,DRBC_opp_L10,RBC_opp_L10,TCHS_opp_L10,SAST_opp_L10,FTAST_opp_L10,PASS_opp_L10,CFGM_opp_L10,CFGA_opp_L10,UFGM_opp_L10,UFGA_opp_L10,DFGM_opp_L10,DFGA_opp_L10,PTS_2PT_MR_opp_L10,PTS_FB_opp_L10,PTS_OFF_TOV_opp_L10,PTS_PAINT_opp_L10,AST_2PM_opp_L10,AST_3PM_opp_L10,UAST_2PM_opp_L10,UAST_3PM_opp_L10,WIN_PCT_L10,COVER_PCT_L10,OREB_PCT_L10,OREB_PCT_opp_L10,DREB_PCT_L10,DREB_PCT_opp_L10,REB_PCT_L10,REB_PCT_opp_L10,TS_PCT_L10,TS_PCT_opp_L10,EFG_PCT_L10,EFG_PCT_opp_L10,AST_RATIO_L10,AST_RATIO_opp_L10,TOV_PCT_L10,TOV_PCT_opp_L10,PIE_L10,FG2M_L20,FG2A_L20,FG3M_L20,FG3A_L20,FTM_L20,FTA_L20,OREB_L20,DREB_L20,AST_L20,STL_L20,BLK_L20,TOV_L20,PF_L20,OFF_RATING_L20,DEF_RATING_L20,PACE_L20,DIST_L20,ORBC_L20,DRBC_L20,RBC_L20,TCHS_L20,SAST_L20,FTAST_L20,PASS_L20,CFGM_L20,CFGA_L20,UFGM_L20,UFGA_L20,DFGM_L20,DFGA_L20,PTS_2PT_MR_L20,PTS_FB_L20,PTS_OFF_TOV_L20,PTS_PAINT_L20,AST_2PM_L20,AST_3PM_L20,UAST_2PM_L20,UAST_3PM_L20,AVG_ATS_DIFF_L20,FG2M_opp_L20,FG2A_opp_L20,FG3M_opp_L20,FG3A_opp_L20,FTM_opp_L20,FTA_opp_L20,OREB_opp_L20,DREB_opp_L20,AST_opp_L20,STL_opp_L20,BLK_opp_L20,TOV_opp_L20,PF_opp_L20,OFF_RATING_opp_L20,DEF_RATING_opp_L20,PACE_opp_L20,DIST_opp_L20,ORBC_opp_L20,DRBC_opp_L20,RBC_opp_L20,TCHS_opp_L20,SAST_opp_L20,FTAST_opp_L20,PASS_opp_L20,CFGM_opp_L20,CFGA_opp_L20,UFGM_opp_L20,UFGA_opp_L20,DFGM_opp_L20,DFGA_opp_L20,PTS_2PT_MR_opp_L20,PTS_FB_opp_L20,PTS_OFF_TOV_opp_L20,PTS_PAINT_opp_L20,AST_2PM_opp_L20,AST_3PM_opp_L20,UAST_2PM_opp_L20,UAST_3PM_opp_L20,WIN_PCT_L20,COVER_PCT_L20,OREB_PCT_L20,OREB_PCT_opp_L20,DREB_PCT_L20,DREB_PCT_opp_L20,REB_PCT_L20,REB_PCT_opp_L20,TS_PCT_L20,TS_PCT_opp_L20,EFG_PCT_L20,EFG_PCT_opp_L20,AST_RATIO_L20,AST_RATIO_opp_L20,TOV_PCT_L20,TOV_PCT_opp_L20,PIE_L20,REST
16146,2013-14,ORL,2013-10-29,0021300001,ORL @ IND,0,87,8.600000,12.5,2.5,1,-10,0,28.723404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.723404,Na

In [304]:
db_filepath = Path.home().joinpath('NBA_model_v1', 'data', 'nba.db')
start_season = '2013-14'
end_season = '2022-23'
connection = sqlite3.connect(db_filepath)
test = pd.read_sql('SELECT * FROM team_stats_ewa_matchup ORDER BY GAME_DATE', con=connection)
tracking = pd.read_sql("SELECT * FROM team_tracking_boxscores", conn)
current_spread_data = pd.read_sql_query("SELECT * FROM spreads", conn)
current_ml_data = pd.read_sql_query("SELECT * FROM moneylines", conn)

team_boxscores = load_team_data(connection, start_season, end_season)
connection.close()

In [305]:
test

,index,SEASON,HOME_TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_HOME_GAME,HOME_TEAM_SCORE,HOME_ML,HOME_SPREAD,HOME_ATS_DIFF,HOME_TEAM_COVERED,HOME_POINT_DIFF,HOME_WL,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_REB_L5,HOME_AST_L5,HOME_STL_L5,HOME_BLK_L5,HOME_TOV_L5,HOME_PF_L5,HOME_PTS_L5,HOME_PLUS_MINUS_L5,HOME_OFF_RATING_L5,HOME_DEF_RATING_L5,HOME_NET_RATING_L5,HOME_PACE_L5,HOME_POSS_L5,HOME_DIST_L5,HOME_ORBC_L5,HOME_DRBC_L5,HOME_RBC_L5,HOME_TCHS_L5,HOME_SAST_L5,HOME_FTAST_L5,HOME_PASS_L5,HOME_CFGM_L5,HOME_CFGA_L5,HOME_UFGM_L5,HOME_UFGA_L5,HOME_DFGM_L5,HOME_DFGA_L5,HOME_PTS_2PT_MR_L5,HOME_PTS_FB_L5,HOME_PTS_OFF_TOV_L5,HOME_PTS_PAINT_L5,HOME_AST_2PM_L5,HOME_AST_3PM_L5,HOME_UAST_2PM_L5,HOME_UAST_3PM_L5,HOME_AVG_ATS_DIFF_L5,HOME_FG2M_opp_L5,HOME_FG2A_opp_L5,HOME_FG3M_opp_L5,HOME_FG3A_opp_L5,HOME_FTM_opp_L5,HOME_FTA_opp_L5,HOME_OREB_opp_L5,HOME_DREB_opp_L5,HOME_REB_opp_L5,HOME_AST_opp_L5,HOME_STL_opp_L5,HOME_BLK_opp_L5,HOME_TOV_opp_L5,HOME_PF_opp_L5,HOME_PTS_opp_L5,HOME_PLUS_MINUS_opp_L5,HOME_OFF_RATING_opp_L5,HOME_DEF_RATING_opp_L5,HOME_NET_RATING_opp_L5,HOME_PACE_opp_L5,HOME_POSS_opp_L5,HOME_DIST_opp_L5,HOME_ORBC_opp_L5,HOME_DRBC_opp_L5,HOME_RBC_opp_L5,HOME_TCHS_opp_L5,HOME_SAST_opp_L5,HOME_FTAST_opp_L5,HOME_PASS_opp_L5,HOME_CFGM_opp_L5,HOME_CFGA_opp_L5,HOME_UFGM_opp_L5,HOME_UFGA_opp_L5,HOME_DFGM_opp_L5,HOME_DFGA_opp_L5,HOME_PTS_2PT_MR_opp_L5,HOME_PTS_FB_opp_L5,HOME_PTS_OFF_TOV_opp_L5,HOME_PTS_PAINT_opp_L5,HOME_AST_2PM_opp_L5,HOME_AST_3PM_opp_L5,HOME_UAST_2PM_opp_L5,HOME_UAST_3PM_opp_L5,HOME_WIN_PCT_L5,HOME_COVER_PCT_L5,HOME_OREB_PCT_L5,HOME_OREB_PCT_opp_L5,HOME_DREB_PCT_L5,HOME_DREB_PCT_opp_L5,HOME_REB_PCT_L5,HOME_REB_PCT_opp_L5,HOME_TS_PCT_L5,HOME_TS_PCT_opp_L5,HOME_EFG_PCT_L5,HOME_EFG_PCT_opp_L5,HOME_AST_RATIO_L5,HOME_AST_RATIO_opp_L5,HOME_TOV_PCT_L5,HOME_TOV_PCT_opp_L5,HOME_PIE_L5,HOME_FG2M_L10,HOME_FG2A_L10,HOME_FG3M_L10,HOME_FG3A_L10,HOME_FTM_L10,HOME_FTA_L10,HOME_OREB_L10,HOME_DREB_L10,HOME_REB_L10,HOME_AST_L10,HOME_STL_L10,HOME_BLK_L10,HOME_TOV_L10,HOME_PF_L10,HOME_PTS_L10,HOME_PLUS_MINUS_L10,HOME_OFF_RATING_L10,HOME_DEF_RATING_L10,HOME_NET_RATING_L10,HOME_PACE_L10,HOME_POSS_L10,HOME_DIST_L10,HOME_ORBC_L10,HOME_DRBC_L10,HOME_RBC_L10,HOME_TCHS_L10,HOME_SAST_L10,HOME_FTAST_L10,HOME_PASS_L10,HOME_CFGM_L10,HOME_CFGA_L10,HOME_UFGM_L10,HOME_UFGA_L10,HOME_DFGM_L10,HOME_DFGA_L10,HOME_PTS_2PT_MR_L10,HOME_PTS_FB_L10,HOME_PTS_OFF_TOV_L10,HOME_PTS_PAINT_L10,HOME_AST_2PM_L10,HOME_AST_3PM_L10,HOME_UAST_2PM_L10,HOME_UAST_3PM_L10,HOME_AVG_ATS_DIFF_L10,HOME_FG2M_opp_L10,HOME_FG2A_opp_L10,HOME_FG3M_opp_L10,HOME_FG3A_opp_L10,HOME_FTM_opp_L10,HOME_FTA_opp_L10,HOME_OREB_opp_L10,HOME_DREB_opp_L10,HOME_REB_opp_L10,HOME_AST_opp_L10,HOME_STL_opp_L10,HOME_BLK_opp_L10,HOME_TOV_opp_L10,HOME_PF_opp_L10,HOME_PTS_opp_L10,HOME_PLUS_MINUS_opp_L10,HOME_OFF_RATING_opp_L10,HOME_DEF_RATING_opp_L10,HOME_NET_RATING_opp_L10,HOME_PACE_opp_L10,HOME_POSS_opp_L10,HOME_DIST_opp_L10,HOME_ORBC_opp_L10,HOME_DRBC_opp_L10,HOME_RBC_opp_L10,HOME_TCHS_opp_L10,HOME_SAST_opp_L10,HOME_FTAST_opp_L10,HOME_PASS_opp_L10,HOME_CFGM_opp_L10,HOME_CFGA_opp_L10,HOME_UFGM_opp_L10,HOME_UFGA_opp_L10,HOME_DFGM_opp_L10,HOME_DFGA_opp_L10,HOME_PTS_2PT_MR_opp_L10,HOME_PTS_FB_opp_L10,HOME_PTS_OFF_TOV_opp_L10,...,AWAY_AST_L10,AWAY_STL_L10,AWAY_BLK_L10,AWAY_TOV_L10,AWAY_PF_L10,AWAY_PTS_L10,AWAY_PLUS_MINUS_L10,AWAY_OFF_RATING_L10,AWAY_DEF_RATING_L10,AWAY_NET_RATING_L10,AWAY_PACE_L10,AWAY_POSS_L10,AWAY_DIST_L10,AWAY_ORBC_L10,AWAY_DRBC_L10,AWAY_RBC_L10,AWAY_TCHS_L10,AWAY_SAST_L10,AWAY_FTAST_L10,AWAY_PASS_L10,AWAY_CFGM_L10,AWAY_CFGA_L10,AWAY_UFGM_L10,AWAY_UFGA_L10,AWAY_DFGM_L10,AWAY_DFGA_L10,AWAY_PTS_2PT_MR_L10,AWAY_PTS_FB_L10,AWAY_PTS_OFF_TOV_L10,AWAY_PTS_PAINT_L10,AWAY_AST_2PM_L10,AWAY_AST_3PM_L10,AWAY_UAST_2PM_L10,AWAY_UAST_3PM_L10,AWAY_AVG_ATS_DIFF_L10,AWAY_FG2M_opp_L10,AWAY_FG2A_opp_L10,AWAY_FG3M_opp_L10,AWAY_FG3A_opp_L10,AWAY_FTM_opp_L10,AWAY_FTA_opp_L10,AWAY_OREB_opp_L10,AWAY_DREB_opp_L10,AWAY_REB_opp_L10,AWAY_AST_opp_L10,AWAY_

In [94]:
lgbr_filepath = Path.home().joinpath('NBA_model_v1', 'models', 'LGBRegressor.sav')
sgdr_filepath = Path.home().joinpath('NBA_model_v1', 'models', 'SGDRegressor_ScorePredictor.sav')
lgbc_filepath = Path.home().joinpath('NBA_model_v1', 'models', 'LGBMClassifier.sav')
sgdc_hinge_filepath = Path.home().joinpath('NBA_model_v1', 'models', 'SGDClassifierHinge_WinPredictor.sav')
sgdc_logloss_filepath = Path.home().joinpath('NBA_model_v1', 'models', 'SGDClassifierLogLoss_WinPredictor.sav')

LGBRegressor = joblib.load(lgbr_filepath)
SGDRegressor = joblib.load(sgdr_filepath)
LGBClassifier = joblib.load(lgbc_filepath)
SGDClassifier_Hinge = joblib.load(sgdc_hinge_filepath)
SGDClassifier_LogLoss = joblib.load(sgdc_logloss_filepath)

In [95]:
print(SGDClassifier_LogLoss)

Pipeline(steps=[('scaler', StandardScaler()),
                ('sgd',
                 SGDClassifier(alpha=0.009368071103349671,
                               l1_ratio=0.455607493192275, loss='log_loss',
                               shuffle=False))])


In [306]:
home_team = 'MIL'
away_team = 'PHI'

row = make_matchup_row(home_team, away_team, df = df_full)

creating matchups between Home and Away team aggregated stats


In [307]:
row

,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_AST_L5,HOME_STL_L5,HOME_BLK_L5,HOME_TOV_L5,HOME_PF_L5,HOME_OFF_RATING_L5,HOME_DEF_RATING_L5,HOME_PACE_L5,HOME_DIST_L5,HOME_ORBC_L5,HOME_DRBC_L5,HOME_RBC_L5,HOME_TCHS_L5,HOME_SAST_L5,HOME_FTAST_L5,HOME_PASS_L5,HOME_CFGM_L5,HOME_CFGA_L5,HOME_UFGM_L5,HOME_UFGA_L5,HOME_DFGM_L5,HOME_DFGA_L5,HOME_PTS_2PT_MR_L5,HOME_PTS_FB_L5,HOME_PTS_OFF_TOV_L5,HOME_PTS_PAINT_L5,HOME_AST_2PM_L5,HOME_AST_3PM_L5,HOME_UAST_2PM_L5,HOME_UAST_3PM_L5,HOME_AVG_ATS_DIFF_L5,HOME_FG2M_opp_L5,HOME_FG2A_opp_L5,HOME_FG3M_opp_L5,HOME_FG3A_opp_L5,HOME_FTM_opp_L5,HOME_FTA_opp_L5,HOME_OREB_opp_L5,HOME_DREB_opp_L5,HOME_AST_opp_L5,HOME_STL_opp_L5,HOME_BLK_opp_L5,HOME_TOV_opp_L5,HOME_PF_opp_L5,HOME_OFF_RATING_opp_L5,HOME_DEF_RATING_opp_L5,HOME_PACE_opp_L5,HOME_DIST_opp_L5,HOME_ORBC_opp_L5,HOME_DRBC_opp_L5,HOME_RBC_opp_L5,HOME_TCHS_opp_L5,HOME_SAST_opp_L5,HOME_FTAST_opp_L5,HOME_PASS_opp_L5,HOME_CFGM_opp_L5,HOME_CFGA_opp_L5,HOME_UFGM_opp_L5,HOME_UFGA_opp_L5,HOME_DFGM_opp_L5,HOME_DFGA_opp_L5,HOME_PTS_2PT_MR_opp_L5,HOME_PTS_FB_opp_L5,HOME_PTS_OFF_TOV_opp_L5,HOME_PTS_PAINT_opp_L5,HOME_AST_2PM_opp_L5,HOME_AST_3PM_opp_L5,HOME_UAST_2PM_opp_L5,HOME_UAST_3PM_opp_L5,HOME_WIN_PCT_L5,HOME_COVER_PCT_L5,HOME_OREB_PCT_L5,HOME_OREB_PCT_opp_L5,HOME_DREB_PCT_L5,HOME_DREB_PCT_opp_L5,HOME_REB_PCT_L5,HOME_REB_PCT_opp_L5,HOME_TS_PCT_L5,HOME_TS_PCT_opp_L5,HOME_EFG_PCT_L5,HOME_EFG_PCT_opp_L5,HOME_AST_RATIO_L5,HOME_AST_RATIO_opp_L5,HOME_TOV_PCT_L5,HOME_TOV_PCT_opp_L5,HOME_PIE_L5,HOME_FG2M_L10,HOME_FG2A_L10,HOME_FG3M_L10,HOME_FG3A_L10,HOME_FTM_L10,HOME_FTA_L10,HOME_OREB_L10,HOME_DREB_L10,HOME_AST_L10,HOME_STL_L10,HOME_BLK_L10,HOME_TOV_L10,HOME_PF_L10,HOME_OFF_RATING_L10,HOME_DEF_RATING_L10,HOME_PACE_L10,HOME_DIST_L10,HOME_ORBC_L10,HOME_DRBC_L10,HOME_RBC_L10,HOME_TCHS_L10,HOME_SAST_L10,HOME_FTAST_L10,HOME_PASS_L10,HOME_CFGM_L10,HOME_CFGA_L10,HOME_UFGM_L10,HOME_UFGA_L10,HOME_DFGM_L10,HOME_DFGA_L10,HOME_PTS_2PT_MR_L10,HOME_PTS_FB_L10,HOME_PTS_OFF_TOV_L10,HOME_PTS_PAINT_L10,HOME_AST_2PM_L10,HOME_AST_3PM_L10,HOME_UAST_2PM_L10,HOME_UAST_3PM_L10,HOME_AVG_ATS_DIFF_L10,HOME_FG2M_opp_L10,HOME_FG2A_opp_L10,HOME_FG3M_opp_L10,HOME_FG3A_opp_L10,HOME_FTM_opp_L10,HOME_FTA_opp_L10,HOME_OREB_opp_L10,HOME_DREB_opp_L10,HOME_AST_opp_L10,HOME_STL_opp_L10,HOME_BLK_opp_L10,HOME_TOV_opp_L10,HOME_PF_opp_L10,HOME_OFF_RATING_opp_L10,HOME_DEF_RATING_opp_L10,HOME_PACE_opp_L10,HOME_DIST_opp_L10,HOME_ORBC_opp_L10,HOME_DRBC_opp_L10,HOME_RBC_opp_L10,HOME_TCHS_opp_L10,HOME_SAST_opp_L10,HOME_FTAST_opp_L10,HOME_PASS_opp_L10,HOME_CFGM_opp_L10,HOME_CFGA_opp_L10,HOME_UFGM_opp_L10,HOME_UFGA_opp_L10,HOME_DFGM_opp_L10,HOME_DFGA_opp_L10,HOME_PTS_2PT_MR_opp_L10,HOME_PTS_FB_opp_L10,HOME_PTS_OFF_TOV_opp_L10,HOME_PTS_PAINT_opp_L10,HOME_AST_2PM_opp_L10,HOME_AST_3PM_opp_L10,HOME_UAST_2PM_opp_L10,HOME_UAST_3PM_opp_L10,HOME_WIN_PCT_L10,HOME_COVER_PCT_L10,HOME_OREB_PCT_L10,HOME_OREB_PCT_opp_L10,HOME_DREB_PCT_L10,HOME_DREB_PCT_opp_L10,HOME_REB_PCT_L10,HOME_REB_PCT_opp_L10,HOME_TS_PCT_L10,HOME_TS_PCT_opp_L10,HOME_EFG_PCT_L10,HOME_EFG_PCT_opp_L10,HOME_AST_RATIO_L10,HOME_AST_RATIO_opp_L10,HOME_TOV_PCT_L10,HOME_TOV_PCT_opp_L10,HOME_PIE_L10,HOME_FG2M_L20,HOME_FG2A_L20,HOME_FG3M_L20,HOME_FG3A_L20,HOME_FTM_L20,HOME_FTA_L20,HOME_OREB_L20,HOME_DREB_L20,HOME_AST_L20,HOME_STL_L20,HOME_BLK_L20,HOME_TOV_L20,...,AWAY_REB_PCT_L5,AWAY_REB_PCT_opp_L5,AWAY_TS_PCT_L5,AWAY_TS_PCT_opp_L5,AWAY_EFG_PCT_L5,AWAY_EFG_PCT_opp_L5,AWAY_AST_RATIO_L5,AWAY_AST_RATIO_opp_L5,AWAY_TOV_PCT_L5,AWAY_TOV_PCT_opp_L5,AWAY_PIE_L5,AWAY_FG2M_L10,AWAY_FG2A_L10,AWAY_FG3M_L10,AWAY_FG3A_L10,AWAY_FTM_L10,AWAY_FTA_L10,AWAY_OREB_L10,AWAY_DREB_L10,AWAY_AST_L10,AWAY_STL_L10,AWAY_BLK_L10,AWAY_TOV_L10,AWAY_PF_L10,AWAY_OFF_RATING_L10,AWAY_DEF_RATING_L10,AWAY_PACE_L10,AWAY_DIST_L10,AWAY_ORBC_L10,AWAY_DRBC_L10,AWAY_RBC_L10,AWAY_TCHS_L10,AWAY_SAST_L10,AWAY_FTAST_L10,AWAY_PASS_L10,AWAY_CFGM_L10,AWAY_CFGA_L10,AWAY_UFGM_L10,AWAY_UFGA_L10,AWAY_DFGM_L10,AWAY_DFGA_L10,AWAY_PTS_2PT_MR_L10,AWAY_PTS_FB_L10,AWA

In [97]:
row

,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_AST_L5,HOME_STL_L5,HOME_BLK_L5,HOME_TOV_L5,HOME_PF_L5,HOME_OFF_RATING_L5,HOME_DEF_RATING_L5,HOME_PACE_L5,HOME_DIST_L5,HOME_ORBC_L5,HOME_DRBC_L5,HOME_RBC_L5,HOME_TCHS_L5,HOME_SAST_L5,HOME_FTAST_L5,HOME_PASS_L5,HOME_CFGM_L5,HOME_CFGA_L5,HOME_UFGM_L5,HOME_UFGA_L5,HOME_DFGM_L5,HOME_DFGA_L5,HOME_PTS_2PT_MR_L5,HOME_PTS_FB_L5,HOME_PTS_OFF_TOV_L5,HOME_PTS_PAINT_L5,HOME_AST_2PM_L5,HOME_AST_3PM_L5,HOME_UAST_2PM_L5,HOME_UAST_3PM_L5,HOME_AVG_ATS_DIFF_L5,HOME_FG2M_opp_L5,HOME_FG2A_opp_L5,HOME_FG3M_opp_L5,HOME_FG3A_opp_L5,HOME_FTM_opp_L5,HOME_FTA_opp_L5,HOME_OREB_opp_L5,HOME_DREB_opp_L5,HOME_AST_opp_L5,HOME_STL_opp_L5,HOME_BLK_opp_L5,HOME_TOV_opp_L5,HOME_PF_opp_L5,HOME_OFF_RATING_opp_L5,HOME_DEF_RATING_opp_L5,HOME_PACE_opp_L5,HOME_DIST_opp_L5,HOME_ORBC_opp_L5,HOME_DRBC_opp_L5,HOME_RBC_opp_L5,HOME_TCHS_opp_L5,HOME_SAST_opp_L5,HOME_FTAST_opp_L5,HOME_PASS_opp_L5,HOME_CFGM_opp_L5,HOME_CFGA_opp_L5,HOME_UFGM_opp_L5,HOME_UFGA_opp_L5,HOME_DFGM_opp_L5,HOME_DFGA_opp_L5,HOME_PTS_2PT_MR_opp_L5,HOME_PTS_FB_opp_L5,HOME_PTS_OFF_TOV_opp_L5,HOME_PTS_PAINT_opp_L5,HOME_AST_2PM_opp_L5,HOME_AST_3PM_opp_L5,HOME_UAST_2PM_opp_L5,HOME_UAST_3PM_opp_L5,HOME_WIN_PCT_L5,HOME_COVER_PCT_L5,HOME_OREB_PCT_L5,HOME_OREB_PCT_opp_L5,HOME_DREB_PCT_L5,HOME_DREB_PCT_opp_L5,HOME_REB_PCT_L5,HOME_REB_PCT_opp_L5,HOME_TS_PCT_L5,HOME_TS_PCT_opp_L5,HOME_EFG_PCT_L5,HOME_EFG_PCT_opp_L5,HOME_AST_RATIO_L5,HOME_AST_RATIO_opp_L5,HOME_TOV_PCT_L5,HOME_TOV_PCT_opp_L5,HOME_PIE_L5,HOME_FG2M_L10,HOME_FG2A_L10,HOME_FG3M_L10,HOME_FG3A_L10,HOME_FTM_L10,HOME_FTA_L10,HOME_OREB_L10,HOME_DREB_L10,HOME_AST_L10,HOME_STL_L10,HOME_BLK_L10,HOME_TOV_L10,HOME_PF_L10,HOME_OFF_RATING_L10,HOME_DEF_RATING_L10,HOME_PACE_L10,HOME_DIST_L10,HOME_ORBC_L10,HOME_DRBC_L10,HOME_RBC_L10,HOME_TCHS_L10,HOME_SAST_L10,HOME_FTAST_L10,HOME_PASS_L10,HOME_CFGM_L10,HOME_CFGA_L10,HOME_UFGM_L10,HOME_UFGA_L10,HOME_DFGM_L10,HOME_DFGA_L10,HOME_PTS_2PT_MR_L10,HOME_PTS_FB_L10,HOME_PTS_OFF_TOV_L10,HOME_PTS_PAINT_L10,HOME_AST_2PM_L10,HOME_AST_3PM_L10,HOME_UAST_2PM_L10,HOME_UAST_3PM_L10,HOME_AVG_ATS_DIFF_L10,HOME_FG2M_opp_L10,HOME_FG2A_opp_L10,HOME_FG3M_opp_L10,HOME_FG3A_opp_L10,HOME_FTM_opp_L10,HOME_FTA_opp_L10,HOME_OREB_opp_L10,HOME_DREB_opp_L10,HOME_AST_opp_L10,HOME_STL_opp_L10,HOME_BLK_opp_L10,HOME_TOV_opp_L10,HOME_PF_opp_L10,HOME_OFF_RATING_opp_L10,HOME_DEF_RATING_opp_L10,HOME_PACE_opp_L10,HOME_DIST_opp_L10,HOME_ORBC_opp_L10,HOME_DRBC_opp_L10,HOME_RBC_opp_L10,HOME_TCHS_opp_L10,HOME_SAST_opp_L10,HOME_FTAST_opp_L10,HOME_PASS_opp_L10,HOME_CFGM_opp_L10,HOME_CFGA_opp_L10,HOME_UFGM_opp_L10,HOME_UFGA_opp_L10,HOME_DFGM_opp_L10,HOME_DFGA_opp_L10,HOME_PTS_2PT_MR_opp_L10,HOME_PTS_FB_opp_L10,HOME_PTS_OFF_TOV_opp_L10,HOME_PTS_PAINT_opp_L10,HOME_AST_2PM_opp_L10,HOME_AST_3PM_opp_L10,HOME_UAST_2PM_opp_L10,HOME_UAST_3PM_opp_L10,HOME_WIN_PCT_L10,HOME_COVER_PCT_L10,HOME_OREB_PCT_L10,HOME_OREB_PCT_opp_L10,HOME_DREB_PCT_L10,HOME_DREB_PCT_opp_L10,HOME_REB_PCT_L10,HOME_REB_PCT_opp_L10,HOME_TS_PCT_L10,HOME_TS_PCT_opp_L10,HOME_EFG_PCT_L10,HOME_EFG_PCT_opp_L10,HOME_AST_RATIO_L10,HOME_AST_RATIO_opp_L10,HOME_TOV_PCT_L10,HOME_TOV_PCT_opp_L10,HOME_PIE_L10,HOME_FG2M_L20,HOME_FG2A_L20,HOME_FG3M_L20,HOME_FG3A_L20,HOME_FTM_L20,HOME_FTA_L20,HOME_OREB_L20,HOME_DREB_L20,HOME_AST_L20,HOME_STL_L20,HOME_BLK_L20,HOME_TOV_L20,...,AWAY_REB_PCT_L5,AWAY_REB_PCT_opp_L5,AWAY_TS_PCT_L5,AWAY_TS_PCT_opp_L5,AWAY_EFG_PCT_L5,AWAY_EFG_PCT_opp_L5,AWAY_AST_RATIO_L5,AWAY_AST_RATIO_opp_L5,AWAY_TOV_PCT_L5,AWAY_TOV_PCT_opp_L5,AWAY_PIE_L5,AWAY_FG2M_L10,AWAY_FG2A_L10,AWAY_FG3M_L10,AWAY_FG3A_L10,AWAY_FTM_L10,AWAY_FTA_L10,AWAY_OREB_L10,AWAY_DREB_L10,AWAY_AST_L10,AWAY_STL_L10,AWAY_BLK_L10,AWAY_TOV_L10,AWAY_PF_L10,AWAY_OFF_RATING_L10,AWAY_DEF_RATING_L10,AWAY_PACE_L10,AWAY_DIST_L10,AWAY_ORBC_L10,AWAY_DRBC_L10,AWAY_RBC_L10,AWAY_TCHS_L10,AWAY_SAST_L10,AWAY_FTAST_L10,AWAY_PASS_L10,AWAY_CFGM_L10,AWAY_CFGA_L10,AWAY_UFGM_L10,AWAY_UFGA_L10,AWAY_DFGM_L10,AWAY_DFGA_L10,AWAY_PTS_2PT_MR_L10,AWAY_PTS_FB_L10,AWA

In [96]:
print(LGBRegressor.predict(row))
print(SGDRegressor.predict(row))
print(LGBClassifier.predict_proba(row))
print(SGDClassifier_Hinge.predict(row))
print(SGDClassifier_LogLoss.predict_proba(row))

[[100.79764099  96.95598337]]
[[101.72996143 101.17784761]]
[[0.58214313 0.41785687]]
[0]
[[0.48772945 0.51227055]]


In [220]:
home_team = 'PHI'
away_team = 'BOS'

row = make_matchup_row(home_team, away_team, df = df_full)
lgbr_model.predict(row)

creating matchups between Home and Away team aggregated stats


array([[102.70060833, 100.81850387]])

In [221]:
home_team = 'LAL'
away_team = 'OKC'

row = make_matchup_row(home_team, away_team, df = df_full)
lgbr_model.predict(row)

creating matchups between Home and Away team aggregated stats


array([[113.77664723, 114.94091434]])

In [198]:
lgbr_model.predict(row)

array([[113.77664723, 114.94091434]])

In [115]:
home_team = 'LAC'
away_team = 'LAL'
start_season = 2013
end_season = 2022

start_season = season_to_string(start_season)
end_season = season_to_string(end_season)

db_filepath = Path.home().joinpath('NBA_model_v1', 'data', 'nba.db')

conn = sqlite3.connect(db_filepath)

print("Loading raw team boxscore data from sql database...")

df = load_team_data(conn, start_season, end_season)
print("Loading betting data from sql database...")
spreads, moneylines = load_betting_data(conn)

print("Cleaning Data...")

df = clean_team_data(df)
df = prep_for_aggregation(df)

clean_mls = clean_moneyline_df(df = moneylines)
clean_spreads = clean_spreads_df(df = spreads)


print("Merging Boxscore and Betting Data...")
merged_df = merge_betting_and_boxscore_data(
    clean_spreads, clean_mls, clean_boxscores = df)


stats_per_100 = normalize_per_100_poss(merged_df)

print("Aggregating over last 5, 10, and 20 game windows")

matchups = create_matchups(stats_per_100)

team_stats_ewa_5 = build_team_avg_stats_df(matchups, span=5)
team_stats_ewa_5 = add_percentage_features(team_stats_ewa_5, span=5)

team_stats_ewa_10 = build_team_avg_stats_df(matchups, span=10)
team_stats_ewa_10 = add_percentage_features(team_stats_ewa_10, span=10)

team_stats_ewa_20 = build_team_avg_stats_df(matchups, span=20)
team_stats_ewa_20 = add_percentage_features(team_stats_ewa_20, span=20)


temp = pd.merge(team_stats_ewa_5, team_stats_ewa_10, how='inner',
                on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                    'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                    'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                    'POINT_DIFF', 'WL'])

df_full = pd.merge(temp, team_stats_ewa_20, how='inner', 
                    on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                        'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                        'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                        'POINT_DIFF', 'WL'])

df_full = df_full.sort_values(['GAME_DATE', 'GAME_ID', 'HOME_GAME'])


columns_to_drop = ['PTS_L5', 'PTS_L10', 'PTS_L20',
                    'PLUS_MINUS_L5', 'PLUS_MINUS_L10', 'PLUS_MINUS_L20',
                    'NET_RATING_L5', 'NET_RATING_L10', 'NET_RATING_L20',
                    'POSS_L5', 'POSS_L10', 'POSS_L20',
                    'REB_L5', 'REB_L10', 'REB_L20',
                    'REB_opp_L5', 'REB_opp_L10', 'REB_opp_L20',
                    'PTS_opp_L5', 'PTS_opp_L10', 'PTS_opp_L20',
                    'PLUS_MINUS_opp_L5', 'PLUS_MINUS_opp_L10', 'PLUS_MINUS_opp_L20',
                    'NET_RATING_opp_L5', 'NET_RATING_opp_L10', 'NET_RATING_opp_L20',
                    'POSS_opp_L5', 'POSS_opp_L10', 'POSS_opp_L20']


df_full = df_full.drop(columns = columns_to_drop)

print("adding rest days")
df_full = add_rest_days(df_full)

print("creating matchups between Home and Away team aggregated stats")

matchup_info_cols = ['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE', 'GAME_ID', 'MATCHUP',
       'HOME_GAME', 'TEAM_SCORE', 'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED',
       'POINT_DIFF', 'WL']

most_recent_home_stats = df_full.loc[df_full['TEAM_ABBREVIATION'] == home_team].tail(1).drop(columns=matchup_info_cols).reset_index(drop=True)
most_recent_home_stats = most_recent_home_stats.add_prefix('HOME_')
most_recent_away_stats = df_full.loc[df_full['TEAM_ABBREVIATION'] == away_team].tail(1).drop(columns=matchup_info_cols).reset_index(drop=True)
most_recent_away_stats = most_recent_away_stats.add_prefix('AWAY_')

matchup_row = pd.concat([most_recent_home_stats, most_recent_away_stats], axis=1)

Loading raw team boxscore data from sql database...
Loading betting data from sql database...
Cleaning Data...
Merging Boxscore and Betting Data...
Aggregating over last 5, 10, and 20 game windows
adding rest days
creating matchups between Home and Away team aggregated stats


In [116]:
matchup_row

,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_AST_L5,HOME_STL_L5,HOME_BLK_L5,HOME_TOV_L5,HOME_PF_L5,HOME_OFF_RATING_L5,HOME_DEF_RATING_L5,HOME_PACE_L5,HOME_DIST_L5,HOME_ORBC_L5,HOME_DRBC_L5,HOME_RBC_L5,HOME_TCHS_L5,HOME_SAST_L5,HOME_FTAST_L5,HOME_PASS_L5,HOME_CFGM_L5,HOME_CFGA_L5,HOME_UFGM_L5,HOME_UFGA_L5,HOME_DFGM_L5,HOME_DFGA_L5,HOME_PTS_2PT_MR_L5,HOME_PTS_FB_L5,HOME_PTS_OFF_TOV_L5,HOME_PTS_PAINT_L5,HOME_AST_2PM_L5,HOME_AST_3PM_L5,HOME_UAST_2PM_L5,HOME_UAST_3PM_L5,HOME_AVG_ATS_DIFF_L5,HOME_FG2M_opp_L5,HOME_FG2A_opp_L5,HOME_FG3M_opp_L5,HOME_FG3A_opp_L5,HOME_FTM_opp_L5,HOME_FTA_opp_L5,HOME_OREB_opp_L5,HOME_DREB_opp_L5,HOME_AST_opp_L5,HOME_STL_opp_L5,HOME_BLK_opp_L5,HOME_TOV_opp_L5,HOME_PF_opp_L5,HOME_OFF_RATING_opp_L5,HOME_DEF_RATING_opp_L5,HOME_PACE_opp_L5,HOME_DIST_opp_L5,HOME_ORBC_opp_L5,HOME_DRBC_opp_L5,HOME_RBC_opp_L5,HOME_TCHS_opp_L5,HOME_SAST_opp_L5,HOME_FTAST_opp_L5,HOME_PASS_opp_L5,HOME_CFGM_opp_L5,HOME_CFGA_opp_L5,HOME_UFGM_opp_L5,HOME_UFGA_opp_L5,HOME_DFGM_opp_L5,HOME_DFGA_opp_L5,HOME_PTS_2PT_MR_opp_L5,HOME_PTS_FB_opp_L5,HOME_PTS_OFF_TOV_opp_L5,HOME_PTS_PAINT_opp_L5,HOME_AST_2PM_opp_L5,HOME_AST_3PM_opp_L5,HOME_UAST_2PM_opp_L5,HOME_UAST_3PM_opp_L5,HOME_WIN_PCT_L5,HOME_COVER_PCT_L5,HOME_OREB_PCT_L5,HOME_OREB_PCT_opp_L5,HOME_DREB_PCT_L5,HOME_DREB_PCT_opp_L5,HOME_REB_PCT_L5,HOME_REB_PCT_opp_L5,HOME_TS_PCT_L5,HOME_TS_PCT_opp_L5,HOME_EFG_PCT_L5,HOME_EFG_PCT_opp_L5,HOME_AST_RATIO_L5,HOME_AST_RATIO_opp_L5,HOME_TOV_PCT_L5,HOME_TOV_PCT_opp_L5,HOME_PIE_L5,HOME_FG2M_L10,HOME_FG2A_L10,HOME_FG3M_L10,HOME_FG3A_L10,HOME_FTM_L10,HOME_FTA_L10,HOME_OREB_L10,HOME_DREB_L10,HOME_AST_L10,HOME_STL_L10,HOME_BLK_L10,HOME_TOV_L10,HOME_PF_L10,HOME_OFF_RATING_L10,HOME_DEF_RATING_L10,HOME_PACE_L10,HOME_DIST_L10,HOME_ORBC_L10,HOME_DRBC_L10,HOME_RBC_L10,HOME_TCHS_L10,HOME_SAST_L10,HOME_FTAST_L10,HOME_PASS_L10,HOME_CFGM_L10,HOME_CFGA_L10,HOME_UFGM_L10,HOME_UFGA_L10,HOME_DFGM_L10,HOME_DFGA_L10,HOME_PTS_2PT_MR_L10,HOME_PTS_FB_L10,HOME_PTS_OFF_TOV_L10,HOME_PTS_PAINT_L10,HOME_AST_2PM_L10,HOME_AST_3PM_L10,HOME_UAST_2PM_L10,HOME_UAST_3PM_L10,HOME_AVG_ATS_DIFF_L10,HOME_FG2M_opp_L10,HOME_FG2A_opp_L10,HOME_FG3M_opp_L10,HOME_FG3A_opp_L10,HOME_FTM_opp_L10,HOME_FTA_opp_L10,HOME_OREB_opp_L10,HOME_DREB_opp_L10,HOME_AST_opp_L10,HOME_STL_opp_L10,HOME_BLK_opp_L10,HOME_TOV_opp_L10,HOME_PF_opp_L10,HOME_OFF_RATING_opp_L10,HOME_DEF_RATING_opp_L10,HOME_PACE_opp_L10,HOME_DIST_opp_L10,HOME_ORBC_opp_L10,HOME_DRBC_opp_L10,HOME_RBC_opp_L10,HOME_TCHS_opp_L10,HOME_SAST_opp_L10,HOME_FTAST_opp_L10,HOME_PASS_opp_L10,HOME_CFGM_opp_L10,HOME_CFGA_opp_L10,HOME_UFGM_opp_L10,HOME_UFGA_opp_L10,HOME_DFGM_opp_L10,HOME_DFGA_opp_L10,HOME_PTS_2PT_MR_opp_L10,HOME_PTS_FB_opp_L10,HOME_PTS_OFF_TOV_opp_L10,HOME_PTS_PAINT_opp_L10,HOME_AST_2PM_opp_L10,HOME_AST_3PM_opp_L10,HOME_UAST_2PM_opp_L10,HOME_UAST_3PM_opp_L10,HOME_WIN_PCT_L10,HOME_COVER_PCT_L10,HOME_OREB_PCT_L10,HOME_OREB_PCT_opp_L10,HOME_DREB_PCT_L10,HOME_DREB_PCT_opp_L10,HOME_REB_PCT_L10,HOME_REB_PCT_opp_L10,HOME_TS_PCT_L10,HOME_TS_PCT_opp_L10,HOME_EFG_PCT_L10,HOME_EFG_PCT_opp_L10,HOME_AST_RATIO_L10,HOME_AST_RATIO_opp_L10,HOME_TOV_PCT_L10,HOME_TOV_PCT_opp_L10,HOME_PIE_L10,HOME_FG2M_L20,HOME_FG2A_L20,HOME_FG3M_L20,HOME_FG3A_L20,HOME_FTM_L20,HOME_FTA_L20,HOME_OREB_L20,HOME_DREB_L20,HOME_AST_L20,HOME_STL_L20,HOME_BLK_L20,HOME_TOV_L20,...,AWAY_REB_PCT_L5,AWAY_REB_PCT_opp_L5,AWAY_TS_PCT_L5,AWAY_TS_PCT_opp_L5,AWAY_EFG_PCT_L5,AWAY_EFG_PCT_opp_L5,AWAY_AST_RATIO_L5,AWAY_AST_RATIO_opp_L5,AWAY_TOV_PCT_L5,AWAY_TOV_PCT_opp_L5,AWAY_PIE_L5,AWAY_FG2M_L10,AWAY_FG2A_L10,AWAY_FG3M_L10,AWAY_FG3A_L10,AWAY_FTM_L10,AWAY_FTA_L10,AWAY_OREB_L10,AWAY_DREB_L10,AWAY_AST_L10,AWAY_STL_L10,AWAY_BLK_L10,AWAY_TOV_L10,AWAY_PF_L10,AWAY_OFF_RATING_L10,AWAY_DEF_RATING_L10,AWAY_PACE_L10,AWAY_DIST_L10,AWAY_ORBC_L10,AWAY_DRBC_L10,AWAY_RBC_L10,AWAY_TCHS_L10,AWAY_SAST_L10,AWAY_FTAST_L10,AWAY_PASS_L10,AWAY_CFGM_L10,AWAY_CFGA_L10,AWAY_UFGM_L10,AWAY_UFGA_L10,AWAY_DFGM_L10,AWAY_DFGA_L10,AWAY_PTS_2PT_MR_L10,AWAY_PTS_FB_L10,AWA

In [354]:
today

[<selenium.webdriver.remote.webelement.WebElement (session="924ff3a19769c1bdecc7b369835e0c1f", element="3b195aee-ea12-4fca-8914-c7990e145c5a")>]

In [355]:
dk_lines_df = get_draftking_lines(date=date.today())
dk_lines_clean = clean_draftking_lines(dk_lines_df)


Scraping Today's Lines


In [356]:
dk_lines_clean

,away_team,home_team,away_spread,home_spread,away_moneyline,home_moneyline,game_date
0,ATL,DET,-7.5,7.5,1.338983,3.450000,2022-10-26
1,ORL,CLE,8.5,-8.5,4.000000,1.273973,2022-10-26
2,BKN,MIL,3.5,-3.5,2.400000,1.606061,2022-10-26
3,PHI,TOR,-2.0,2.0,1.769231,2.100000,2022-10-26
4,CHA,NYK,7.5,-7.5,3.500000,1.333333,2022-10-26
5,SAS,MIN,9.0,-9.0,4.200000,1.256410,2022-10-26
6,IND,CHI,7.5,-7.5,3.500000,1.333333,2022-10-26
7,HOU,UTA,7.5,-7.5,3.450000,1.338983,2022-10-26
8,LAL,DEN,5.5,-5.5,2.800000,1.476190,2022-10-26
9,MIA,POR,-3.0,3.0,1.689655,2.250000,2022-10-26


In [387]:
start_season = 2013
end_season = 2022


db_filepath = Path.home().joinpath('NBA_model_v1', 'data', 'nba.db')

conn = sqlite3.connect(db_filepath)

print("Loading raw team boxscore data from sql database...")

df = load_team_data(conn, start_season, end_season)
print("Loading betting data from sql database...")
spreads, moneylines = load_betting_data(conn)

print("Cleaning Data...")

df = clean_team_data(df)
df = prep_for_aggregation(df)

clean_mls = clean_moneyline_df(df = moneylines)
clean_spreads = clean_spreads_df(df = spreads)


print("Merging Boxscore and Betting Data...")
merged_df = merge_betting_and_boxscore_data(clean_spreads, clean_mls, clean_boxscores = df)

# today = date.today()
# print("Getting Draftking Lines...") 


team_abbreviation = []
game_id = []
matchup = []
home_game = []

for i, row in dk_lines_clean.iterrows():
    home_team = row['home_team']
    away_team = row['away_team']
    
    team_abbreviation.append(home_team)
    home_game.append(1)
    game_id.append(i)
    matchup.append(home_team + ' vs. ' + away_team)
    
    team_abbreviation.append(away_team)
    home_game.append(0)
    game_id.append(i)
    matchup.append(away_team + ' @ ' + home_team)
    

todays_matchups = pd.DataFrame({'TEAM_ABBREVIATION':team_abbreviation,
                                'MATCHUP':matchup,
                                'GAME_ID':game_id,
                                'HOME_GAME':home_game})

todays_matchups['SEASON'] = '2022-23'
todays_matchups['GAME_DATE'] = pd.to_datetime(date.today())

merged_df_with_todays_games = pd.concat([merged_df, todays_matchups])

merged_df_with_todays_games

stats_per_100 = normalize_per_100_poss(merged_df_with_todays_games)

matchups = create_matchups(stats_per_100)


# matchups
print("Aggregating over last 5, 10, and 20 game windows")
    

team_stats_ewa_5 = build_team_avg_stats_df(matchups, span=5)
team_stats_ewa_5 = add_percentage_features(team_stats_ewa_5, span=5)

team_stats_ewa_10 = build_team_avg_stats_df(matchups, span=10)
team_stats_ewa_10 = add_percentage_features(team_stats_ewa_10, span=10)

team_stats_ewa_20 = build_team_avg_stats_df(matchups, span=20)
team_stats_ewa_20 = add_percentage_features(team_stats_ewa_20, span=20)


temp = pd.merge(team_stats_ewa_5, team_stats_ewa_10, how='inner',
                on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                    'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                    'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                    'POINT_DIFF', 'WL'])

df_full = pd.merge(temp, team_stats_ewa_20, how='inner', 
                    on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                        'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                        'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                        'POINT_DIFF', 'WL'])

df_full = df_full.sort_values(['GAME_DATE', 'GAME_ID', 'HOME_GAME'])


print("adding rest days")
df_full = add_rest_days_adv(df_full)

print("creating matchups between Home and Away team aggregated stats")
df_full = make_matchups_2(df_full)

print("Resorting by date")
df_full = df_full.sort_values(['GAME_DATE', 'GAME_ID', 'HOME_HOME_GAME'])


Loading raw team boxscore data from sql database...
Loading betting data from sql database...
Cleaning Data...
Merging Boxscore and Betting Data...


,SEASON,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,TEAM_SCORE,WL,POINT_DIFF,HOME_GAME,RECORD,FG2M,FG2A,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,OFF_RATING,DEF_RATING,NET_RATING,PACE,POSS,DIST,ORBC,DRBC,RBC,TCHS,SAST,FTAST,PASS,CFGM,CFGA,UFGM,UFGA,DFGM,DFGA,PTS_2PT_MR,PTS_FB,PTS_OFF_TOV,PTS_PAINT,AST_2PM,AST_3PM,UAST_2PM,UAST_3PM,ML,SPREAD,ATS_DIFF,TEAM_COVERED
0,2013-14,1610612746,LAC,Los Angeles Clippers,0021300003,2013-10-29,LAC @ LAL,103.0,0.0,-13.0,0,0.0,33.0,62.0,8.0,21.0,13.0,23.0,10.0,30.0,40.0,27.0,11.0,4.0,16.0,21.0,103.0,-13.0,105.1,117.2,-12.1,98.5,98.0,17.44,40.0,64.0,98.0,394.0,0.0,2.0,279.0,23.0,44.0,18.0,39.0,12.0,21.0,18.0,18.0,25.0,47.0,19.0,7.0,13.0,1.0,1.156250,-10.0,-23.0,0.0
1,2013-14,1610612747,LAL,Los Angeles Lakers,0021300003,2013-10-29,LAL vs. LAC,116.0,1.0,13.0,1,1.0,28.0,64.0,14.0,29.0,18.0,28.0,18.0,34.0,52.0,23.0,8.0,6.0,19.0,23.0,116.0,13.0,117.2,105.1,12.1,98.5,99.0,17.35,44.0,64.0,103.0,449.0,4.0,0.0,318.0,15.0,31.0,27.0,62.0,20.0,28.0,14.0,12.0,27.0,41.0,12.0,11.0,15.0,2.0,5.550000,10.0,23.0,1.0
2,2013-14,1610612741,CHI,Chicago Bulls,0021300002,2013-10-29,CHI @ MIA,95.0,0.0,-12.0,0,0.0,28.0,57.0,7.0,26.0,18.0,23.0,11.0,30.0,41.0,23.0,11.0,4.0,19.0,27.0,95.0,-12.0,96.9,108.1,-11.1,98.5,98.0,17.64,37.0,57.0,87.0,446.0,3.0,3.0,317.0,17.0,35.0,18.0,48.0,11.0,20.0,5.0,17.0,30.0,49.0,15.0,7.0,12.0,0.0,2.850000,5.0,-7.0,0.0
3,2013-14,1610612748,MIA,Miami Heat,0021300002,2013-10-29,MIA vs. CHI,107.0,1.0,12.0,1,1.0,26.0,52.0,11.0,20.0,22.0,29.0,5.0,35.0,40.0,26.0,10.0,7.0,20.0,21.0,107.0,12.0,108.1,96.9,11.1,98.5,99.0,17.01,22.0,73.0,93.0,462.0,3.0,3.0,340.0,11.0,28.0,26.0,44.0,17.0,26.0,17.0,9.0,14.0,34.0,15.0,11.0,10.0,0.0,1.454545,-5.0,7.0,1.0
4,2013-14,1610612754,IND,Indiana Pacers,0021300001,2013-10-29,IND vs. ORL,97.0,1.0,10.0,1,1.0,27.0,54.0,7.0,17.0,22.0,32.0,10.0,34.0,44.0,17.0,4.0,18.0,21.0,13.0,97.0,10.0,103.2,92.6,10.6,94.0,94.0,16.09,40.0,78.0,117.0,428.0,1.0,1.0,310.0,20.0,38.0,14.0,33.0,9.0,21.0,18.0,12.0,29.0,35.0,11.0,4.0,15.0,2.0,1.090498,-12.5,-2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,2022-23,NaN,HOU,NaN,7,2022-10-26,HOU @ UTA,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2022-23,NaN,DEN,NaN,8,2022-10-26,DEN vs. LAL,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,2022-23,NaN,LAL,NaN,8,2022-10-26,LAL @ DEN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,2022-23,NaN,POR,NaN,9,2022-10-26,POR vs. MIA,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [390]:
stats_per_100 = normalize_per_100_poss(merged_df_with_todays_games)

matchups = create_matchups(stats_per_100)


# matchups
print("Aggregating over last 5, 10, and 20 game windows")
    

team_stats_ewa_5 = build_team_avg_stats_df(matchups, span=5)
team_stats_ewa_5 = add_percentage_features(team_stats_ewa_5, span=5)

team_stats_ewa_10 = build_team_avg_stats_df(matchups, span=10)
team_stats_ewa_10 = add_percentage_features(team_stats_ewa_10, span=10)

team_stats_ewa_20 = build_team_avg_stats_df(matchups, span=20)
team_stats_ewa_20 = add_percentage_features(team_stats_ewa_20, span=20)


temp = pd.merge(team_stats_ewa_5, team_stats_ewa_10, how='inner',
                on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                    'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                    'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                    'POINT_DIFF', 'WL'])

df_full = pd.merge(temp, team_stats_ewa_20, how='inner', 
                    on=['SEASON', 'TEAM_ABBREVIATION', 'GAME_DATE',
                        'GAME_ID', 'MATCHUP', 'HOME_GAME', 'TEAM_SCORE',
                        'ML', 'SPREAD', 'ATS_DIFF', 'TEAM_COVERED', 
                        'POINT_DIFF', 'WL'])

df_full = df_full.sort_values(['GAME_DATE', 'GAME_ID', 'HOME_GAME'])


print("adding rest days")
df_full = add_rest_days_adv(df_full)

print("creating matchups between Home and Away team aggregated stats")
df_full = make_matchups_2(df_full)

print("Resorting by date")
df_full = df_full.sort_values(['GAME_DATE', 'GAME_ID', 'HOME_HOME_GAME'])

Aggregating over last 5, 10, and 20 game windows
adding rest days
creating matchups between Home and Away team aggregated stats
Resorting by date


In [393]:
df_full.loc[df_full['HOME_TEAM_ABBREVIATION'] == 'GSW']

,SEASON,HOME_TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_HOME_GAME,HOME_TEAM_SCORE,HOME_ML,HOME_SPREAD,HOME_ATS_DIFF,HOME_TEAM_COVERED,HOME_POINT_DIFF,HOME_WL,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_REB_L5,HOME_AST_L5,HOME_STL_L5,HOME_BLK_L5,HOME_TOV_L5,HOME_PF_L5,HOME_PTS_L5,HOME_PLUS_MINUS_L5,HOME_OFF_RATING_L5,HOME_DEF_RATING_L5,HOME_NET_RATING_L5,HOME_PACE_L5,HOME_POSS_L5,HOME_DIST_L5,HOME_ORBC_L5,HOME_DRBC_L5,HOME_RBC_L5,HOME_TCHS_L5,HOME_SAST_L5,HOME_FTAST_L5,HOME_PASS_L5,HOME_CFGM_L5,HOME_CFGA_L5,HOME_UFGM_L5,HOME_UFGA_L5,HOME_DFGM_L5,HOME_DFGA_L5,HOME_PTS_2PT_MR_L5,HOME_PTS_FB_L5,HOME_PTS_OFF_TOV_L5,HOME_PTS_PAINT_L5,HOME_AST_2PM_L5,HOME_AST_3PM_L5,HOME_UAST_2PM_L5,HOME_UAST_3PM_L5,HOME_AVG_ATS_DIFF_L5,HOME_FG2M_opp_L5,HOME_FG2A_opp_L5,HOME_FG3M_opp_L5,HOME_FG3A_opp_L5,HOME_FTM_opp_L5,HOME_FTA_opp_L5,HOME_OREB_opp_L5,HOME_DREB_opp_L5,HOME_REB_opp_L5,HOME_AST_opp_L5,HOME_STL_opp_L5,HOME_BLK_opp_L5,HOME_TOV_opp_L5,HOME_PF_opp_L5,HOME_PTS_opp_L5,HOME_PLUS_MINUS_opp_L5,HOME_OFF_RATING_opp_L5,HOME_DEF_RATING_opp_L5,HOME_NET_RATING_opp_L5,HOME_PACE_opp_L5,HOME_POSS_opp_L5,HOME_DIST_opp_L5,HOME_ORBC_opp_L5,HOME_DRBC_opp_L5,HOME_RBC_opp_L5,HOME_TCHS_opp_L5,HOME_SAST_opp_L5,HOME_FTAST_opp_L5,HOME_PASS_opp_L5,HOME_CFGM_opp_L5,HOME_CFGA_opp_L5,HOME_UFGM_opp_L5,HOME_UFGA_opp_L5,HOME_DFGM_opp_L5,HOME_DFGA_opp_L5,HOME_PTS_2PT_MR_opp_L5,HOME_PTS_FB_opp_L5,HOME_PTS_OFF_TOV_opp_L5,HOME_PTS_PAINT_opp_L5,HOME_AST_2PM_opp_L5,HOME_AST_3PM_opp_L5,HOME_UAST_2PM_opp_L5,HOME_UAST_3PM_opp_L5,HOME_WIN_PCT_L5,HOME_COVER_PCT_L5,HOME_OREB_PCT_L5,HOME_OREB_PCT_opp_L5,HOME_DREB_PCT_L5,HOME_DREB_PCT_opp_L5,HOME_REB_PCT_L5,HOME_REB_PCT_opp_L5,HOME_TS_PCT_L5,HOME_TS_PCT_opp_L5,HOME_EFG_PCT_L5,HOME_EFG_PCT_opp_L5,HOME_AST_RATIO_L5,HOME_AST_RATIO_opp_L5,HOME_TOV_PCT_L5,HOME_TOV_PCT_opp_L5,HOME_PIE_L5,HOME_FG2M_L10,HOME_FG2A_L10,HOME_FG3M_L10,HOME_FG3A_L10,HOME_FTM_L10,HOME_FTA_L10,HOME_OREB_L10,HOME_DREB_L10,HOME_REB_L10,HOME_AST_L10,HOME_STL_L10,HOME_BLK_L10,HOME_TOV_L10,HOME_PF_L10,HOME_PTS_L10,HOME_PLUS_MINUS_L10,HOME_OFF_RATING_L10,HOME_DEF_RATING_L10,HOME_NET_RATING_L10,HOME_PACE_L10,HOME_POSS_L10,HOME_DIST_L10,HOME_ORBC_L10,HOME_DRBC_L10,HOME_RBC_L10,HOME_TCHS_L10,HOME_SAST_L10,HOME_FTAST_L10,HOME_PASS_L10,HOME_CFGM_L10,HOME_CFGA_L10,HOME_UFGM_L10,HOME_UFGA_L10,HOME_DFGM_L10,HOME_DFGA_L10,HOME_PTS_2PT_MR_L10,HOME_PTS_FB_L10,HOME_PTS_OFF_TOV_L10,HOME_PTS_PAINT_L10,HOME_AST_2PM_L10,HOME_AST_3PM_L10,HOME_UAST_2PM_L10,HOME_UAST_3PM_L10,HOME_AVG_ATS_DIFF_L10,HOME_FG2M_opp_L10,HOME_FG2A_opp_L10,HOME_FG3M_opp_L10,HOME_FG3A_opp_L10,HOME_FTM_opp_L10,HOME_FTA_opp_L10,HOME_OREB_opp_L10,HOME_DREB_opp_L10,HOME_REB_opp_L10,HOME_AST_opp_L10,HOME_STL_opp_L10,HOME_BLK_opp_L10,HOME_TOV_opp_L10,HOME_PF_opp_L10,HOME_PTS_opp_L10,HOME_PLUS_MINUS_opp_L10,HOME_OFF_RATING_opp_L10,HOME_DEF_RATING_opp_L10,HOME_NET_RATING_opp_L10,HOME_PACE_opp_L10,HOME_POSS_opp_L10,HOME_DIST_opp_L10,HOME_ORBC_opp_L10,HOME_DRBC_opp_L10,HOME_RBC_opp_L10,HOME_TCHS_opp_L10,HOME_SAST_opp_L10,HOME_FTAST_opp_L10,HOME_PASS_opp_L10,HOME_CFGM_opp_L10,HOME_CFGA_opp_L10,HOME_UFGM_opp_L10,HOME_UFGA_opp_L10,HOME_DFGM_opp_L10,HOME_DFGA_opp_L10,HOME_PTS_2PT_MR_opp_L10,HOME_PTS_FB_opp_L10,HOME_PTS_OFF_TOV_opp_L10,HOME_PTS_PAINT_opp_L10,...,AWAY_BLK_L10,AWAY_TOV_L10,AWAY_PF_L10,AWAY_PTS_L10,AWAY_PLUS_MINUS_L10,AWAY_OFF_RATING_L10,AWAY_DEF_RATING_L10,AWAY_NET_RATING_L10,AWAY_PACE_L10,AWAY_POSS_L10,AWAY_DIST_L10,AWAY_ORBC_L10,AWAY_DRBC_L10,AWAY_RBC_L10,AWAY_TCHS_L10,AWAY_SAST_L10,AWAY_FTAST_L10,AWAY_PASS_L10,AWAY_CFGM_L10,AWAY_CFGA_L10,AWAY_UFGM_L10,AWAY_UFGA_L10,AWAY_DFGM_L10,AWAY_DFGA_L10,AWAY_PTS_2PT_MR_L10,AWAY_PTS_FB_L10,AWAY_PTS_OFF_TOV_L10,AWAY_PTS_PAINT_L10,AWAY_AST_2PM_L10,AWAY_AST_3PM_L10,AWAY_UAST_2PM_L10,AWAY_UAST_3PM_L10,AWAY_AVG_ATS_DIFF_L10,AWAY_FG2M_opp_L10,AWAY_FG2A_opp_L10,AWAY_FG3M_opp_L10,AWAY_FG3A_opp_L10,AWAY_FTM_opp_L10,AWAY_FTA_opp_L10,AWAY_OREB_opp_L10,AWAY_DREB_opp_L10,AWAY_REB_opp_L10,AWAY_AST_opp_L10,AWAY_STL_opp_L

In [370]:
todays_matchups

,TEAM_ABBREVIATION,MATCHUP,HOME_GAME,SEASON,GAME_DATE
0,DET,DET vs. ATL,1,2022-23,2022-10-26
1,ATL,ATL @ DET,0,2022-23,2022-10-26
2,CLE,CLE vs. ORL,1,2022-23,2022-10-26
3,ORL,ORL @ CLE,0,2022-23,2022-10-26
4,MIL,MIL vs. BKN,1,2022-23,2022-10-26
5,BKN,BKN @ MIL,0,2022-23,2022-10-26
6,TOR,TOR vs. PHI,1,2022-23,2022-10-26
7,PHI,PHI @ TOR,0,2022-23,2022-10-26
8,NYK,NYK vs. CHA,1,2022-23,2022-10-26
9,CHA,CHA @ NYK,0,2022-23,2022-10-26


In [404]:
from src.data.process_data_no_split import get_data_from_db_all

X, y, df = get_data_from_db_all(target=['HOME_TEAM_SCORE', 'AWAY_TEAM_SCORE'], db_filepath=db_filepath)



C:\Users\Jordan Nishimura\NBA_model_v1\data\nba.db


In [408]:
dk_lines_clean.to_csv('../src/models/dk_lines_clean_20221026.csv', index=False)

In [405]:
X

,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_AST_L5,HOME_STL_L5,HOME_BLK_L5,HOME_TOV_L5,HOME_PF_L5,HOME_OFF_RATING_L5,HOME_DEF_RATING_L5,HOME_PACE_L5,HOME_DIST_L5,HOME_ORBC_L5,HOME_DRBC_L5,HOME_RBC_L5,HOME_TCHS_L5,HOME_SAST_L5,HOME_FTAST_L5,HOME_PASS_L5,HOME_CFGM_L5,HOME_CFGA_L5,HOME_UFGM_L5,HOME_UFGA_L5,HOME_DFGM_L5,HOME_DFGA_L5,HOME_PTS_2PT_MR_L5,HOME_PTS_FB_L5,HOME_PTS_OFF_TOV_L5,HOME_PTS_PAINT_L5,HOME_AST_2PM_L5,HOME_AST_3PM_L5,HOME_UAST_2PM_L5,HOME_UAST_3PM_L5,HOME_AVG_ATS_DIFF_L5,HOME_FG2M_opp_L5,HOME_FG2A_opp_L5,HOME_FG3M_opp_L5,HOME_FG3A_opp_L5,HOME_FTM_opp_L5,HOME_FTA_opp_L5,HOME_OREB_opp_L5,HOME_DREB_opp_L5,HOME_AST_opp_L5,HOME_STL_opp_L5,HOME_BLK_opp_L5,HOME_TOV_opp_L5,HOME_PF_opp_L5,HOME_OFF_RATING_opp_L5,HOME_DEF_RATING_opp_L5,HOME_PACE_opp_L5,HOME_DIST_opp_L5,HOME_ORBC_opp_L5,HOME_DRBC_opp_L5,HOME_RBC_opp_L5,HOME_TCHS_opp_L5,HOME_SAST_opp_L5,HOME_FTAST_opp_L5,HOME_PASS_opp_L5,HOME_CFGM_opp_L5,HOME_CFGA_opp_L5,HOME_UFGM_opp_L5,HOME_UFGA_opp_L5,HOME_DFGM_opp_L5,HOME_DFGA_opp_L5,HOME_PTS_2PT_MR_opp_L5,HOME_PTS_FB_opp_L5,HOME_PTS_OFF_TOV_opp_L5,HOME_PTS_PAINT_opp_L5,HOME_AST_2PM_opp_L5,HOME_AST_3PM_opp_L5,HOME_UAST_2PM_opp_L5,HOME_UAST_3PM_opp_L5,HOME_WIN_PCT_L5,HOME_COVER_PCT_L5,HOME_OREB_PCT_L5,HOME_OREB_PCT_opp_L5,HOME_DREB_PCT_L5,HOME_DREB_PCT_opp_L5,HOME_REB_PCT_L5,HOME_REB_PCT_opp_L5,HOME_TS_PCT_L5,HOME_TS_PCT_opp_L5,HOME_EFG_PCT_L5,HOME_EFG_PCT_opp_L5,HOME_AST_RATIO_L5,HOME_AST_RATIO_opp_L5,HOME_TOV_PCT_L5,HOME_TOV_PCT_opp_L5,HOME_PIE_L5,HOME_FG2M_L10,HOME_FG2A_L10,HOME_FG3M_L10,HOME_FG3A_L10,HOME_FTM_L10,HOME_FTA_L10,HOME_OREB_L10,HOME_DREB_L10,HOME_AST_L10,HOME_STL_L10,HOME_BLK_L10,HOME_TOV_L10,HOME_PF_L10,HOME_OFF_RATING_L10,HOME_DEF_RATING_L10,HOME_PACE_L10,HOME_DIST_L10,HOME_ORBC_L10,HOME_DRBC_L10,HOME_RBC_L10,HOME_TCHS_L10,HOME_SAST_L10,HOME_FTAST_L10,HOME_PASS_L10,HOME_CFGM_L10,HOME_CFGA_L10,HOME_UFGM_L10,HOME_UFGA_L10,HOME_DFGM_L10,HOME_DFGA_L10,HOME_PTS_2PT_MR_L10,HOME_PTS_FB_L10,HOME_PTS_OFF_TOV_L10,HOME_PTS_PAINT_L10,HOME_AST_2PM_L10,HOME_AST_3PM_L10,HOME_UAST_2PM_L10,HOME_UAST_3PM_L10,HOME_AVG_ATS_DIFF_L10,HOME_FG2M_opp_L10,HOME_FG2A_opp_L10,HOME_FG3M_opp_L10,HOME_FG3A_opp_L10,HOME_FTM_opp_L10,HOME_FTA_opp_L10,HOME_OREB_opp_L10,HOME_DREB_opp_L10,HOME_AST_opp_L10,HOME_STL_opp_L10,HOME_BLK_opp_L10,HOME_TOV_opp_L10,HOME_PF_opp_L10,HOME_OFF_RATING_opp_L10,HOME_DEF_RATING_opp_L10,HOME_PACE_opp_L10,HOME_DIST_opp_L10,HOME_ORBC_opp_L10,HOME_DRBC_opp_L10,HOME_RBC_opp_L10,HOME_TCHS_opp_L10,HOME_SAST_opp_L10,HOME_FTAST_opp_L10,HOME_PASS_opp_L10,HOME_CFGM_opp_L10,HOME_CFGA_opp_L10,HOME_UFGM_opp_L10,HOME_UFGA_opp_L10,HOME_DFGM_opp_L10,HOME_DFGA_opp_L10,HOME_PTS_2PT_MR_opp_L10,HOME_PTS_FB_opp_L10,HOME_PTS_OFF_TOV_opp_L10,HOME_PTS_PAINT_opp_L10,HOME_AST_2PM_opp_L10,HOME_AST_3PM_opp_L10,HOME_UAST_2PM_opp_L10,HOME_UAST_3PM_opp_L10,HOME_WIN_PCT_L10,HOME_COVER_PCT_L10,HOME_OREB_PCT_L10,HOME_OREB_PCT_opp_L10,HOME_DREB_PCT_L10,HOME_DREB_PCT_opp_L10,HOME_REB_PCT_L10,HOME_REB_PCT_opp_L10,HOME_TS_PCT_L10,HOME_TS_PCT_opp_L10,HOME_EFG_PCT_L10,HOME_EFG_PCT_opp_L10,HOME_AST_RATIO_L10,HOME_AST_RATIO_opp_L10,HOME_TOV_PCT_L10,HOME_TOV_PCT_opp_L10,HOME_PIE_L10,HOME_FG2M_L20,HOME_FG2A_L20,HOME_FG3M_L20,HOME_FG3A_L20,HOME_FTM_L20,HOME_FTA_L20,HOME_OREB_L20,HOME_DREB_L20,HOME_AST_L20,HOME_STL_L20,HOME_BLK_L20,HOME_TOV_L20,...,AWAY_REB_PCT_L5,AWAY_REB_PCT_opp_L5,AWAY_TS_PCT_L5,AWAY_TS_PCT_opp_L5,AWAY_EFG_PCT_L5,AWAY_EFG_PCT_opp_L5,AWAY_AST_RATIO_L5,AWAY_AST_RATIO_opp_L5,AWAY_TOV_PCT_L5,AWAY_TOV_PCT_opp_L5,AWAY_PIE_L5,AWAY_FG2M_L10,AWAY_FG2A_L10,AWAY_FG3M_L10,AWAY_FG3A_L10,AWAY_FTM_L10,AWAY_FTA_L10,AWAY_OREB_L10,AWAY_DREB_L10,AWAY_AST_L10,AWAY_STL_L10,AWAY_BLK_L10,AWAY_TOV_L10,AWAY_PF_L10,AWAY_OFF_RATING_L10,AWAY_DEF_RATING_L10,AWAY_PACE_L10,AWAY_DIST_L10,AWAY_ORBC_L10,AWAY_DRBC_L10,AWAY_RBC_L10,AWAY_TCHS_L10,AWAY_SAST_L10,AWAY_FTAST_L10,AWAY_PASS_L10,AWAY_CFGM_L10,AWAY_CFGA_L10,AWAY_UFGM_L10,AWAY_UFGA_L10,AWAY_DFGM_L10,AWAY_DFGA_L10,AWAY_PTS_2PT_MR_L10,AWAY_PTS_FB_L10,AWA

In [399]:
connection = sqlite3.connect(db_path)
test = pd.read_sql("""SELECT * FROM team_stats_ewa_matchup_dev""", connection)

In [403]:
test.isnull()

,index,SEASON,HOME_TEAM_ABBREVIATION,GAME_DATE,GAME_ID,MATCHUP,HOME_HOME_GAME,HOME_TEAM_SCORE,HOME_ML,HOME_SPREAD,HOME_ATS_DIFF,HOME_TEAM_COVERED,HOME_POINT_DIFF,HOME_WL,HOME_FG2M_L5,HOME_FG2A_L5,HOME_FG3M_L5,HOME_FG3A_L5,HOME_FTM_L5,HOME_FTA_L5,HOME_OREB_L5,HOME_DREB_L5,HOME_REB_L5,HOME_AST_L5,HOME_STL_L5,HOME_BLK_L5,HOME_TOV_L5,HOME_PF_L5,HOME_PTS_L5,HOME_PLUS_MINUS_L5,HOME_OFF_RATING_L5,HOME_DEF_RATING_L5,HOME_NET_RATING_L5,HOME_PACE_L5,HOME_POSS_L5,HOME_DIST_L5,HOME_ORBC_L5,HOME_DRBC_L5,HOME_RBC_L5,HOME_TCHS_L5,HOME_SAST_L5,HOME_FTAST_L5,HOME_PASS_L5,HOME_CFGM_L5,HOME_CFGA_L5,HOME_UFGM_L5,HOME_UFGA_L5,HOME_DFGM_L5,HOME_DFGA_L5,HOME_PTS_2PT_MR_L5,HOME_PTS_FB_L5,HOME_PTS_OFF_TOV_L5,HOME_PTS_PAINT_L5,HOME_AST_2PM_L5,HOME_AST_3PM_L5,HOME_UAST_2PM_L5,HOME_UAST_3PM_L5,HOME_AVG_ATS_DIFF_L5,HOME_FG2M_opp_L5,HOME_FG2A_opp_L5,HOME_FG3M_opp_L5,HOME_FG3A_opp_L5,HOME_FTM_opp_L5,HOME_FTA_opp_L5,HOME_OREB_opp_L5,HOME_DREB_opp_L5,HOME_REB_opp_L5,HOME_AST_opp_L5,HOME_STL_opp_L5,HOME_BLK_opp_L5,HOME_TOV_opp_L5,HOME_PF_opp_L5,HOME_PTS_opp_L5,HOME_PLUS_MINUS_opp_L5,HOME_OFF_RATING_opp_L5,HOME_DEF_RATING_opp_L5,HOME_NET_RATING_opp_L5,HOME_PACE_opp_L5,HOME_POSS_opp_L5,HOME_DIST_opp_L5,HOME_ORBC_opp_L5,HOME_DRBC_opp_L5,HOME_RBC_opp_L5,HOME_TCHS_opp_L5,HOME_SAST_opp_L5,HOME_FTAST_opp_L5,HOME_PASS_opp_L5,HOME_CFGM_opp_L5,HOME_CFGA_opp_L5,HOME_UFGM_opp_L5,HOME_UFGA_opp_L5,HOME_DFGM_opp_L5,HOME_DFGA_opp_L5,HOME_PTS_2PT_MR_opp_L5,HOME_PTS_FB_opp_L5,HOME_PTS_OFF_TOV_opp_L5,HOME_PTS_PAINT_opp_L5,HOME_AST_2PM_opp_L5,HOME_AST_3PM_opp_L5,HOME_UAST_2PM_opp_L5,HOME_UAST_3PM_opp_L5,HOME_WIN_PCT_L5,HOME_COVER_PCT_L5,HOME_OREB_PCT_L5,HOME_OREB_PCT_opp_L5,HOME_DREB_PCT_L5,HOME_DREB_PCT_opp_L5,HOME_REB_PCT_L5,HOME_REB_PCT_opp_L5,HOME_TS_PCT_L5,HOME_TS_PCT_opp_L5,HOME_EFG_PCT_L5,HOME_EFG_PCT_opp_L5,HOME_AST_RATIO_L5,HOME_AST_RATIO_opp_L5,HOME_TOV_PCT_L5,HOME_TOV_PCT_opp_L5,HOME_PIE_L5,HOME_FG2M_L10,HOME_FG2A_L10,HOME_FG3M_L10,HOME_FG3A_L10,HOME_FTM_L10,HOME_FTA_L10,HOME_OREB_L10,HOME_DREB_L10,HOME_REB_L10,HOME_AST_L10,HOME_STL_L10,HOME_BLK_L10,HOME_TOV_L10,HOME_PF_L10,HOME_PTS_L10,HOME_PLUS_MINUS_L10,HOME_OFF_RATING_L10,HOME_DEF_RATING_L10,HOME_NET_RATING_L10,HOME_PACE_L10,HOME_POSS_L10,HOME_DIST_L10,HOME_ORBC_L10,HOME_DRBC_L10,HOME_RBC_L10,HOME_TCHS_L10,HOME_SAST_L10,HOME_FTAST_L10,HOME_PASS_L10,HOME_CFGM_L10,HOME_CFGA_L10,HOME_UFGM_L10,HOME_UFGA_L10,HOME_DFGM_L10,HOME_DFGA_L10,HOME_PTS_2PT_MR_L10,HOME_PTS_FB_L10,HOME_PTS_OFF_TOV_L10,HOME_PTS_PAINT_L10,HOME_AST_2PM_L10,HOME_AST_3PM_L10,HOME_UAST_2PM_L10,HOME_UAST_3PM_L10,HOME_AVG_ATS_DIFF_L10,HOME_FG2M_opp_L10,HOME_FG2A_opp_L10,HOME_FG3M_opp_L10,HOME_FG3A_opp_L10,HOME_FTM_opp_L10,HOME_FTA_opp_L10,HOME_OREB_opp_L10,HOME_DREB_opp_L10,HOME_REB_opp_L10,HOME_AST_opp_L10,HOME_STL_opp_L10,HOME_BLK_opp_L10,HOME_TOV_opp_L10,HOME_PF_opp_L10,HOME_PTS_opp_L10,HOME_PLUS_MINUS_opp_L10,HOME_OFF_RATING_opp_L10,HOME_DEF_RATING_opp_L10,HOME_NET_RATING_opp_L10,HOME_PACE_opp_L10,HOME_POSS_opp_L10,HOME_DIST_opp_L10,HOME_ORBC_opp_L10,HOME_DRBC_opp_L10,HOME_RBC_opp_L10,HOME_TCHS_opp_L10,HOME_SAST_opp_L10,HOME_FTAST_opp_L10,HOME_PASS_opp_L10,HOME_CFGM_opp_L10,HOME_CFGA_opp_L10,HOME_UFGM_opp_L10,HOME_UFGA_opp_L10,HOME_DFGM_opp_L10,HOME_DFGA_opp_L10,HOME_PTS_2PT_MR_opp_L10,HOME_PTS_FB_opp_L10,HOME_PTS_OFF_TOV_opp_L10,...,AWAY_BLK_L10,AWAY_TOV_L10,AWAY_PF_L10,AWAY_PTS_L10,AWAY_PLUS_MINUS_L10,AWAY_OFF_RATING_L10,AWAY_DEF_RATING_L10,AWAY_NET_RATING_L10,AWAY_PACE_L10,AWAY_POSS_L10,AWAY_DIST_L10,AWAY_ORBC_L10,AWAY_DRBC_L10,AWAY_RBC_L10,AWAY_TCHS_L10,AWAY_SAST_L10,AWAY_FTAST_L10,AWAY_PASS_L10,AWAY_CFGM_L10,AWAY_CFGA_L10,AWAY_UFGM_L10,AWAY_UFGA_L10,AWAY_DFGM_L10,AWAY_DFGA_L10,AWAY_PTS_2PT_MR_L10,AWAY_PTS_FB_L10,AWAY_PTS_OFF_TOV_L10,AWAY_PTS_PAINT_L10,AWAY_AST_2PM_L10,AWAY_AST_3PM_L10,AWAY_UAST_2PM_L10,AWAY_UAST_3PM_L10,AWAY_AVG_ATS_DIFF_L10,AWAY_FG2M_opp_L10,AWAY_FG2A_opp_L10,AWAY_FG3M_opp_L10,AWAY_FG3A_opp_L10,AWAY_FTM_opp_L10,AWAY_FTA_opp_L10,AWAY_OREB_opp_L10,AWAY_DREB_opp_L10,AWAY_REB_opp_L10,AWAY_AST_opp_L10,AWAY_STL_opp_L10,AWAY_BLK_opp_L